<a id='top'></a>

# Watershed metrics for all Huc12 subwatersheds that intersect AWC recorded streams
Iterate over AKSSF regions and identify all HUC12 sub-watersheds that intersect an AWC recorded stream. Identify the downstream-most/outlet catchment for each Huc12 from this pool and convert the polygon to INSIDE centroid point.  Calculate the distance to coastline as the straight line distance in Km from centroid point to NHD recorded coastline and export this as a feature class/table.  Next use the outlet catchments unique identifier to query the appropriate dataset and build watersheds for each outlet catchment.  Calculate watershed metrics listed in the covariate section and export final merged csv using the catchment unique identifier field "cat_ID_con" to link the metric back to the source catchment/HUC12.  Merge watersheds together and use to calculate covariates.
## Covariates
Covariates needed for prediction on AWC-HUC12 outlets are as follows:
### Summer Precipitation
To be calculated in R using the outlet catchment centroid point feature class exported during outlet identification process.
### Watershed Slope Metrics
Regional Slope grids created in AKSSF_merge_grids.ipynb script.
Run zonal statistics on slope grid using merged watershed as zone feature.
Field names and descriptions:
* **awc_huc12s_wtd_slope_mn = mean watershed slope**
* **awc_huc12s_wtd_slope_min = min watershed slope**
* **awc_huc12s_wtd_slope_max = max watershed slope**
* **awc_huc12s_wtd_slope_sd (or cv) = standard deviation of watershed slope**
### Watershed Percent North Aspect
Regional North grids created in AKSSF_merge_grids.ipynb scripts.
North = aspects from 315-45 degrees and calculate the percentage of land area facing north for each watershed. Run tabulate area on north grid using merged watershed as zone feature and calculate percentage from area.
Field names and descriptions:
* **awc_huc12s_north_wtd = percent watershed with north aspect**
### Watershed Percent Lake Cover
Lakes feature classes for each network datatype (NHDPlus vs TauDEM) stored in AKSSF hydrography database on the T:
Calculate percentage of watershed that is covered by lakes/ponds using tabulate interesection between lake features and watersheds.
Field names and descriptions:
* **awc_huc12s_wtd_lake_per = percent watershed covered by lakes**
### Watershed Percent Glacier Cover
Use input glacier fc (from previous covariate calculations) stored in regional gdbs an calculate percent of watershed with glacial coverage using tabulate intersection between lake features and watersheds.
Field names and descriptions:
* **awc_huc12s_wtd_glac_per = percent watershed covered by glaciers**
### Watershed LCLD
LCLD rasters created in AKSSF_MODIS_lcld_ipynb.
Iterate over LCLD input rasters to produce yearly means for watersheds using zonal statistics.
Field names and descriptions:
* **awc_huc12s_wtd_lcld_mn_YYYY = mean lcld**


## Import modules
Set initial environments and import modules
Print system paths

In [1]:
import os, arcpy, sys,datetime
arcpy.env.overwriteOutput = True
sr = arcpy.SpatialReference(3338)  #'NAD_1983_Alaska_Albers'
arcpy.env.outputCoordinateSystem = sr

print('imports complete')
print(f'{("-"*100)}')
print(f'sys paths {sys.path}')
print(f'{("-"*100)}')
print(f'Python Environment set to - {sys.base_exec_prefix}')
print(f'{("-"*100)}')
print (datetime.datetime.now())
outdir = os.path.dirname(os.getcwd())
print(f'CSV table output directory set to {outdir}')

imports complete
----------------------------------------------------------------------------------------------------
sys paths ['C:\\Users\\dwmerrigan\\Documents\\GitHub\\AKSSF\\data_preparation\\sensitivity_drivers\\landcover', 'C:\\Users\\dwmerrigan\\AppData\\Local\\Programs\\ArcGIS\\Pro\\Resources\\ArcPy', 'C:\\Users\\dwmerrigan\\Documents\\GitHub\\AKSSF', 'C:\\Users\\dwmerrigan\\AppData\\Local\\Programs\\ArcGIS\\Pro\\bin\\Python\\envs\\arcgispro-py3\\python37.zip', 'C:\\Users\\dwmerrigan\\AppData\\Local\\Programs\\ArcGIS\\Pro\\bin\\Python\\envs\\arcgispro-py3\\DLLs', 'C:\\Users\\dwmerrigan\\AppData\\Local\\Programs\\ArcGIS\\Pro\\bin\\Python\\envs\\arcgispro-py3\\lib', 'C:\\Users\\dwmerrigan\\AppData\\Local\\Programs\\ArcGIS\\Pro\\bin\\Python\\envs\\arcgispro-py3', '', 'C:\\Users\\dwmerrigan\\AppData\\Local\\Programs\\ArcGIS\\Pro\\bin\\Python\\envs\\arcgispro-py3\\lib\\site-packages', 'C:\\Users\\dwmerrigan\\AppData\\Local\\Programs\\ArcGIS\\Pro\\bin', 'C:\\Users\\dwmerrigan\\AppDa

## Functions
Define any functions that will be used

In [2]:
# Function to add key, value pairs to dictionary
def append_value(dict_obj, key, value):
    # Check if key exist in dict or not
    if key in dict_obj:
        # Key exist in dict.
        # Check if type of value of key is list or not
        if not isinstance(dict_obj[key], list):
            # If type is not list then make it list
            dict_obj[key] = [dict_obj[key]]
        # Append the value in list
        dict_obj[key].append(value)
    else:
        # As key is not in dict,
        # so, add key-value pair
        dict_obj[key] = value
# Function to remove parenthesis from user inputs
def replace_all(userinput, dic):
    for i, j in dic.items():
        userinput = userinput.replace(i, j)
    return userinput



## Section 1
### Set input datasets, output locations, and scratch workspaces
User to input paths for necessary input data and output locations
Scratch workspaces and output workspaces will be automatically created if they do not already exist.

In [21]:
# Get user inputs
# Used to format user inputs
inputDict = {"'":"",'"':""}

# Specify path to nhdPlus lakes
while True:
    try:
        userinput7 = replace_all((input('Input path to NHDPlus lakes feature class.\nHydrography database on T: has copy\nLeave blank and hit enter to use the default location.\nDefault = D:\\Basedata\\AKSSF_Basedata\\AKSSF_Basedata.gdb\\AKSSF_NHDPlus_LakePond_alb') or 'D:\\Basedata\\AKSSF_Basedata\\AKSSF_Basedata.gdb\\AKSSF_NHDPlus_LakePond_alb'),inputDict)
        if not arcpy.Exists(userinput7):
            print('Path specified does not exist!\nPlease re-enter a valid path')
            continue
        else:
            nhd_lakes_fc = userinput7
            break
    except KeyboardInterrupt:
        print('interrupted!')
        sys.exit()

print(f'NHDPlus lakes set to {nhd_lakes_fc}\n {"-"*100}')

# Specify path to nhd lakes for tau regions
while True:
    try:
        userinput8 = replace_all((input('Input path to NHD_H_Alaska_State_GDB lakes feature class.\nHydrography database on T: has copy\nLeave blank and hit enter to use the default location.\nDefault = D:\\Basedata\\AKSSF_Basedata\\AKSSF_Basedata.gdb\\AKSSF_NHD_LakesPonds_alb') or 'D:\\Basedata\\AKSSF_Basedata\\AKSSF_Basedata.gdb\\AKSSF_NHD_LakesPonds_alb'),inputDict)
        if not arcpy.Exists(userinput8):
            print('Path specified does not exist!\nPlease re-enter a valid path')
            continue
        else:
            tau_lakes_fc = userinput8
            break
    except KeyboardInterrupt:
        print('interrupted!')
        sys.exit()
print(f'NHD_H_Alaska lakes for TauDEM regions set to {nhd_lakes_fc}\n {"-"*100}')

# Specify path to AKSSF parent directory
while True:
    try:
        userinput = replace_all((input('Input AKSSF parent directory containing regional sub-folders.\nLeave blank and hit enter to use the default location.\nDefault = D:\\GIS\\AKSSF\\') or 'D:\\GIS\\AKSSF'),inputDict)
        if not arcpy.Exists(userinput):
            print('Path specified does not exist!\nPlease re-enter a valid path')
            continue
        else:
            data_dir = userinput
            break
    except KeyboardInterrupt:
        print('interrupted!')
        sys.exit()

print(f'AKSSF parent directory set to {data_dir}\n {"-"*100}')

# Specify path to AWC events fc
while True:
    try:
        userinput2 = replace_all((input('Input path to awc events feature class or shapefile.\nLeave blank and hit enter to use the default location.\nDefault = D:\\Basedata\\AWC\\AWC_2021_SpeciesEvents.gdb\\awcEventArcs') or "D:\\Basedata\\AWC\\AWC_2021_SpeciesEvents.gdb\\awcEventArcs"), inputDict)
        if not arcpy.Exists(userinput2):
            print('Path specified does not exist!\nPlease re-enter a valid path')
            continue
        else:
            awc_events = userinput2
            break
    except KeyboardInterrupt:
        print('interrupted!')
        sys.exit()
print(f'AWC events feature class set to {awc_events}\n {"-"*100}')

# Enter output destination  - to create working folders and gdbs
while True:
    try:
        userinput3 = replace_all((input('Input path to create working folders.\nLeave blank and hit enter to use the default location.\nDefault = D:\\GIS\\') or 'D:\\GIS\\'),inputDict)
        if not arcpy.Exists(userinput):
            print('Path specified does not exist!\nPlease re-enter a valid path')
            continue
        else:
            temp_path = userinput3
            print(f'Output locations will be created at {temp_path}\n {"-"*100}')
            break
    except KeyboardInterrupt:
        print('interrupted!')
        sys.exit()


# Path to lcld rasters
lcld_folder = r'D:\\Basedata\\LCLD_rasters_archive'
# Enter output destination  - to create working folders and gdbs
while True:
    try:
        userinput4 = replace_all((input('Input path to LCLD raster parent folder.\nLeave blank and hit enter to use the default location.\nDefault = D:\\Basedata\\LCLD_rasters_archive\\') or 'D:\\Basedata\\LCLD_rasters_archive'),inputDict)
        if not arcpy.Exists(userinput):
            print('Path specified does not exist!\nPlease re-enter a valid path')
            continue
        else:
            lcld_folder = userinput4
            print(f'LCLD subfolders located at {lcld_folder}\n {"-"*100}')
            break
    except KeyboardInterrupt:
        print('interrupted!')
        sys.exit()

## Create working output location to store intermediate data
dirname = 'AKSSF_awcHuc12_cv'
tempgdbname = 'AKSSF_awcHuc12_cv.gdb'
temp_dir = os.path.join(temp_path, dirname)

# Create temporary working gdb
if not arcpy.Exists(temp_dir):
    os.makedirs(temp_dir)
else:
    print(f'Working Folder already created {temp_dir}\n {"-"*100}')

outcheck = os.path.join(temp_dir, tempgdbname)

if arcpy.Exists(outcheck):
    print (f'Output location already exists {outcheck}\n {"-"*100}')
    outgdb = outcheck
if not arcpy.Exists(outcheck):
    print(f'Creating output GDB\n {"-"*100}')
    tempgdb = arcpy.CreateFileGDB_management(temp_dir,tempgdbname)
    print (f'Output geodatabase created at {outcheck}\n {"-"*100}')
    outgdb = tempgdb.getOutput(0)


NHDPlus lakes set to D:\Basedata\AKSSF_Basedata\AKSSF_Basedata.gdb\AKSSF_NHDPlus_LakePond_alb
 ----------------------------------------------------------------------------------------------------
NHD_H_Alaska lakes for TauDEM regions set to D:\Basedata\AKSSF_Basedata\AKSSF_Basedata.gdb\AKSSF_NHDPlus_LakePond_alb
 ----------------------------------------------------------------------------------------------------
AKSSF parent directory set to D:\GIS\AKSSF
 ----------------------------------------------------------------------------------------------------
AWC events feature class set to D:\Basedata\AWC\AWC_2021_SpeciesEvents.gdb\awcEventArcs
 ----------------------------------------------------------------------------------------------------
Output locations will be created at D:\GIS\
 ----------------------------------------------------------------------------------------------------
LCLD subfolders located at D:\Basedata\LCLD_rasters_archive
 ------------------------------------------

## Section 1.1
### Set and create local copies of additional input data


In [4]:
import arcpy
arcpy.env.overwriteOutput = True
sr = arcpy.SpatialReference(3338)  #'NAD_1983_Alaska_Albers'
arcpy.env.outputCoordinateSystem = sr

nhdplusfol = []
tahuc12=[]

# Create and set HUC12 data if it does not already exist
nhdplushucs = os.path.join(outgdb, 'NHDPlusHUC12')
tauhucs = os.path.join(outgdb, 'NHD_H_HUC12')

if not arcpy.Exists(tauhucs):
    print(f'Huc12 data for Tau Regions not yet created')
    #Enter path to WBDHU12 from NHD_H gdb
    while True:
        try:
            userinput6 = replace_all((input('Input path to source WBDHU12 for state of Alaska.\nLeave blank and hit enter to use the default location.\nDefault = D:\\Basedata\\NHD_H_Alaska_State_GDB.gdb\\WBD\\WBDHU12') or 'D:\\Basedata\\NHD_H_Alaska_State_GDB.gdb\\WBD\\WBDHU12'),inputDict)
            if not arcpy.Exists(userinput6):
                print('Path specified does not exist!\nPlease re-enter a valid path')
                continue
            else:
                tauhuc12 = userinput6
                arcpy.CopyFeatures_management(tauhuc12,tauhucs)
                print(f'WBD Huc12  copied to {tauhucs}\n {"-"*100}')
                break
        except KeyboardInterrupt:
            print('interrupted!')
            sys.exit()

else:
    print(f'Tau Region Hucs {tauhucs} located and exists = {arcpy.Exists(tauhucs)}')

if not arcpy.Exists(nhdplushucs):
    print(f'Huc12 data for NHDPlus Regions not yet created')
    #Enter NHDplus data folder
    while True:
        try:
            userinput5 = replace_all((input('Input path to source NHDPlus parent folder.\nLeave blank and hit enter to use the default location.\nDefault = D:\\Basedata\\NHDPlus') or 'D:\\Basedata\\NHDPlus'),inputDict)
            if not arcpy.Exists(userinput5):
                print('Path specified does not exist!\nPlease re-enter a valid path')
                continue
            else:
                nhdplusfol = userinput5
                print(f'NHD HUC12 will be copied to {nhdplushucs}\n {"-"*100}')
                hucs = []
                walk = arcpy.da.Walk(nhdplusfol, datatype="FeatureClass", type="Polygon")

                for dirpath, dirnames, filenames in walk:
                    for filename in filenames:
                        if filename == 'WBDHU12':
                            hucs.append(os.path.join(dirpath, filename))
                arcpy.Merge_management(hucs,nhdplushucs,'','ADD_SOURCE_INFO')
                break
        except KeyboardInterrupt:
            print('interrupted!')
            sys.exit()
else:
    print(f'NHDPlus Hucs {nhdplushucs} located and exists = {arcpy.Exists(nhdplushucs)}')



Tau Region Hucs D:\GIS\AKSSF_awcHuc12_cv\AKSSF_awcHuc12_cv.gdb\NHD_H_HUC12 located and exists = True
NHDPlus Hucs D:\GIS\AKSSF_awcHuc12_cv\AKSSF_awcHuc12_cv.gdb\NHDPlusHUC12 located and exists = True


## Section 2
### By Region
Identify downstream-most catchment for each Huc 12
 * Select by location and select catchment with most us contributing area
    * NHDPlus
        * Use update cursor to join TotalDrainageAreaSqKm from vaa table to catchment
        * Find max value from selection and save as outlet catchment for that HUC12
    * TauDEM
        * DSContArea - Drainage area at the downstream end of the link. Generally this is one grid cell upstream of the downstream end because the drainage area at the downstream end grid cell includes the area of the stream being joined.
 * Generate Centroid point and append to centroid dataset
    * Retain cat_id and Huc12-id
 * Append to HUC12 catchment dataset

In [10]:
import arcpy, time, os, datetime, operator

arcpy.env.workspace = data_dir
regions = arcpy.ListWorkspaces()

# Dictionaries and lists
nhdplusoutlets = []
tauoutlets = []
nhdplusawccatouts = []
tauawccatouts = []
vaaDict = {}
strDict = {}
catsDict = {}
huc12Dict = {}
nhdidDict = {}
tauidDict = {}
tauhuc12Dict = {}

# Separate data by source type
nhdplus_dat = ['Cook_Inlet','Copper_River']
tauDem_dat = ['Bristol_Bay', 'Kodiak', 'Prince_William_Sound']

# Loop through all processing areas
# rois = nhdplus_dat + tauDem_dat

# Or comment above and specify below specific subset
#regions = ['D:\\GIS\\AKSSF\\Cook_Inlet', 'D:\\GIS\\AKSSF\\Copper_River' ]
regions = ['Prince_William_Sound']
# Start timing function
processStart = time.time()
processStartdt = datetime.datetime.now()

for region in regions:
    roi = os.path.basename(region)
    print(roi)
    if roi in nhdplus_dat:
        # Start roi time
        roi_start = time.time()
        hucs = nhdplushucs
        catsList = []
        outletList = []
        print(f'{roi} using data from {region} folder')
        # Set workspace to region folder
        arcpy.env.workspace = region
        gdb = arcpy.ListWorkspaces(workspace_type='FileGDB')
        sourcegdb = gdb[0]
        walk = arcpy.da.Walk(sourcegdb, datatype = ['FeatureClass','Table'])
        for dirpath, dirnames, filenames in walk:
            for filename in filenames:
                if filename == 'cats_merge':
                    cats  = os.path.join(dirpath, filename)
                    append_value(catsDict,roi,cats)
                elif filename == 'vaa_merge':
                    vaas = os.path.join(dirpath, filename)
                    append_value(vaaDict, roi, vaas)
        #Output names and paths
        outletcatsname = roi + '_NhdAwcH12_cats_outlets'
        outcatspath = os.path.join(outgdb,outletcatsname)
        outcatspath2 = os.path.join(sourcegdb,'awc_huc12_catchment_outlets')
        outletcatptsname = roi + '_NhdAwcH12_cats_outlets_pts'
        outcatptspath = os.path.join(outgdb,outletcatptsname)
        outcatptspath2 = os.path.join(sourcegdb,'awc_huc12_catchment_outlets_pts')

        if not arcpy.Exists(outcatspath):
            # Build Value dictionary to relate NHDPlus id to contributing area
            fields = ['NHDPlusID','TotDASqKm']
            fields2 = fields + ['cat_ID_con']
            valueDict = {int(r[0]):(r[1]) for r in arcpy.da.SearchCursor(vaas, fields)}
            where_clause=f'"MERGE_SRC" LIKE \'%{roi}%\''
            print(f'where_clause = {where_clause}')
            huclayer = arcpy.MakeFeatureLayer_management(hucs,'huclayer',where_clause = where_clause)
            print(f'{arcpy.GetCount_management(huclayer)} huc12s in {roi}')
            print(('*'*100))
            hucselect = arcpy.SelectLayerByLocation_management(huclayer,'INTERSECT',awc_events,'','SUBSET_SELECTION')
            print(('*'*100))
            print(f'{arcpy.GetCount_management(hucselect)} Huc12s in {roi} intersect awc events input')
            print(('*'*100))
            hucFields = [f for f in arcpy.ListFields(hucselect)]
            vcount =1
            with arcpy.da.SearchCursor(hucselect,['HUC12','SHAPE@']) as cur:
                for row in cur:
                    print(f'Processing HUC {row[0]}')
                    inhuc = row[1]
                    cat_layer = arcpy.MakeFeatureLayer_management(cats,'cat_layer')
                    # Select by location using awc and huc 12
                    arcpy.SelectLayerByLocation_management(cat_layer,'HAVE_THEIR_CENTER_IN',inhuc,'','NEW_SELECTION')
                    print(f'{vcount}. Finding outlet for HUC {row[0]} out of {arcpy.GetCount_management(cat_layer)} catchments.\n{("*" * 60)}')
                    catList = [r[0] for r in arcpy.da.SearchCursor(cat_layer, 'NHDPlusID')]
                    intersect = list(set(catList).intersection(valueDict))
                    catDict = {int(i):(valueDict[i]) for i in intersect}
                    # Find Catchment with max drainage area
                    outcatch = max(catDict.items(), key = operator.itemgetter(1))[0]
                    append_value(huc12Dict, row[0], [int(outcatch),roi,valueDict[int(outcatch)]])
                    append_value(nhdidDict,int(outcatch),[roi,row[0], valueDict[int(outcatch)]])
                    outletList.append(int(outcatch))
                    vcount+=1
                del(row)
            del(cur)

            outlet_cats = arcpy.MakeFeatureLayer_management(cats,'outlet_cats')
            out_expression ='"NHDPlusID" IN ' + str(tuple(outletList))
            #print(out_expression)
            outlet_cats_select = arcpy.SelectLayerByAttribute_management(outlet_cats,'NEW_SELECTION', out_expression)
            print(f'Creating copy of {arcpy.GetCount_management(outlet_cats)} outlet catchments for Region {roi} at {outcatspath}')
            print(('*'*100))

            # Copy outputs
            arcpy.FeatureClassToFeatureClass_conversion(outlet_cats_select,outgdb,outletcatsname)
            arcpy.FeatureToPoint_management(outcatspath, outcatptspath, 'INSIDE')
            # Create Copies to akssf data_dir regional gdbs also
            arcpy.FeatureClassToFeatureClass_conversion(outlet_cats_select,sourcegdb,'awc_huc12_catchment_outlets')
            arcpy.FeatureToPoint_management(outcatspath2, outcatptspath2, 'INSIDE')
            nhdplusoutlets.append(outcatptspath)
            nhdplusawccatouts.append(outcatspath)
            # Add total drainage km from value dict to feature classes and cat_ID_con from regDict
            upfcs = [outcatspath, outcatptspath,outcatptspath2,outcatptspath2]
            for upfc in upfcs:
                arcpy.AddField_management(upfc,fields[1],'TEXT')
                arcpy.AddField_management(upfc,fields2[2],'TEXT')
                with arcpy.da.UpdateCursor(upfc,fields2) as cur:
                    for row in cur:
                        row[1] = valueDict[row[0]]
                        row[2] = roi + '_' + str(int(row[0]))
                        cur.updateRow(row)
                    del(row)
                del(cur)

            # End roi time
            roi_stop = time.time()
            roi_time = int (roi_stop - roi_start)
            print(f'{roi} Elapsed time: ({datetime.timedelta(seconds=roi_time)})')
            print(f'{"*"*60}')
        else:
            print(f'Catchments for {roi} already created at {outcatspath2}')

    elif roi in tauDem_dat:
        # Start roi time
        roi_start = time.time()
        hucs = tauhucs
        catsList = []
        outletList = []
        print(f'{roi} using data from {region} folder')
        # Set workspace to region folder
        arcpy.env.workspace = region
        gdb = arcpy.ListWorkspaces(workspace_type='FileGDB')
        sourcegdb = gdb[0]
        walk = arcpy.da.Walk(sourcegdb, datatype = ['FeatureClass','Table'])
        for dirpath, dirnames, filenames in walk:
            for filename in filenames:
                if filename == 'cats_merge':
                    cats  = os.path.join(dirpath, filename)
                    append_value(catsDict,roi,cats)
                elif filename == 'streams_merge':
                    streams = os.path.join(dirpath, filename)
                    append_value(strDict, roi, streams)

        #Output names and paths
        outletcatsname = roi + '_TauAwcH12_cats_outlets'
        outcatspath = os.path.join(outgdb,outletcatsname)
        outcatspath2 = os.path.join(sourcegdb,'awc_huc12_catchment_outlets')
        outletcatptsname = roi + '_TauAwcH12_cats_outlets_pts'
        outcatptspath = os.path.join(outgdb,outletcatptsname)
        outcatptspath2 = os.path.join(sourcegdb,'awc_huc12_catchment_outlets_pts')
        print(('-'*100),'\n')
        if not arcpy.Exists(outcatspath):
            # Build Value dictionary to relate NHDPlus id to contributing area
            fields = ['LINKNO','DSContArea']
            fields2 = fields + ['cat_ID_con']
            fields3 = ['gridcode','DSContArea','cat_ID_con']
            valueDict = {int(r[0]):(r[1]) for r in arcpy.da.SearchCursor(streams, fields)}
            huclayer = arcpy.MakeFeatureLayer_management(hucs,'huclayer')
            hucselect_reg = arcpy.SelectLayerByLocation_management(huclayer,'INTERSECT',streams,'','NEW_SELECTION')
            print(f'{arcpy.GetCount_management(huclayer)} huc12s in {roi}')
            print(('*'*100))
            hucselect = arcpy.SelectLayerByLocation_management(hucselect_reg,'INTERSECT',awc_events,'','SUBSET_SELECTION')
            print(('*'*100))
            print(f'{arcpy.GetCount_management(hucselect)} Huc12s in {roi} intersect awc events input')
            print(('*'*100))
            hucFields = [f for f in arcpy.ListFields(hucselect)]
            vcount =1
            with arcpy.da.SearchCursor(hucselect,['HUC12','SHAPE@']) as cur:
                for row in cur:
                    print(f'Processing HUC {row[0]}')
                    inhuc = row[1]
                    cat_layer = arcpy.MakeFeatureLayer_management(cats,'cat_layer')
                    # Select by location using awc and huc 12
                    arcpy.SelectLayerByLocation_management(cat_layer,'HAVE_THEIR_CENTER_IN',inhuc,'','NEW_SELECTION')
                    print(f'{vcount}. Finding outlet for HUC {row[0]} out of {arcpy.GetCount_management(cat_layer)} catchments.\n{("*" * 60)}')
                    catList = [r[0] for r in arcpy.da.SearchCursor(cat_layer, 'gridcode')]
                    intersect = list(set(catList).intersection(valueDict))
                    catDict = {int(i):(valueDict[i]) for i in intersect}
                    # Find Catchment with max drainage area
                    outcatch = max(catDict.items(), key = operator.itemgetter(1))[0]
                    append_value(tauhuc12Dict, row[0], [int(outcatch),roi,valueDict[int(outcatch)]])
                    append_value(tauidDict,int(outcatch),[roi,row[0], valueDict[int(outcatch)]])
                    outletList.append(int(outcatch))
                    vcount+=1
                del(row)
            del(cur)
            outlet_cats = arcpy.MakeFeatureLayer_management(cats,'outlet_cats')
            out_expression ='"gridcode" IN ' + str(tuple(outletList))
            #print(out_expression)
            outlet_cats_select = arcpy.SelectLayerByAttribute_management(outlet_cats,'NEW_SELECTION', out_expression)
            print(f'Creating copy of {arcpy.GetCount_management(outlet_cats)} outlet catchments for Region {roi} at {outcatspath}')
            print(('*'*100))

            # Copy outputs
            arcpy.FeatureClassToFeatureClass_conversion(outlet_cats_select,outgdb,outletcatsname)
            arcpy.FeatureToPoint_management(outcatspath, outcatptspath, 'INSIDE')
            # Create Copies to akssf data_dir regional gdbs also
            arcpy.FeatureClassToFeatureClass_conversion(outlet_cats_select,sourcegdb,'awc_huc12_catchment_outlets')
            arcpy.FeatureToPoint_management(outcatspath2, outcatptspath2, 'INSIDE')
            tauoutlets.append(outcatptspath)
            tauawccatouts.append(outcatspath)
            # Add total drainage km from value dict to feature classes and cat_ID_con from regDict
            upfcs = [outcatspath, outcatptspath,outcatptspath2,outcatptspath2]
            for upfc in upfcs:
                arcpy.AddField_management(upfc,fields[1],'TEXT')
                arcpy.AddField_management(upfc,fields2[2],'TEXT')
                with arcpy.da.UpdateCursor(upfc,fields3) as cur:
                    for row in cur:
                        row[1] = valueDict[row[0]]
                        row[2] = roi + '_' + str(int(row[0]))
                        cur.updateRow(row)
                    del(row)
                del(cur)

            # End roi time
            roi_stop = time.time()
            roi_time = int (roi_stop - roi_start)
            print(f'{roi} Elapsed time: ({datetime.timedelta(seconds=roi_time)})')
            print(f'{"*"*60}')

# End timing
processEnd = time.time()
processElapsed = int(processEnd - processStart)
processSuccess_time = datetime.datetime.now()
print(f'Process complete')

# Report success
print(f'Process completed at {processSuccess_time.strftime("%Y-%m-%d %H:%M")} '
      f'(Elapsed time: {datetime.timedelta(seconds=processElapsed)})')
print(f'{"*"*100}')

Prince_William_Sound
Prince_William_Sound using data from Prince_William_Sound folder
----------------------------------------------------------------------------------------------------
Region Prince_William_Sound will not be processed
---------------------------------------------------------------------------------------------------- 

158 huc12s in Prince_William_Sound
****************************************************************************************************
****************************************************************************************************
112 Huc12s in Prince_William_Sound intersect awc events input
****************************************************************************************************
Processing HUC 190202011602
1. Finding outlet for HUC 190202011602 out of 253 catchments.
************************************************************
Processing HUC 190202011604
2. Finding outlet for HUC 190202011604 out of 105 catchments.
*******************

## Section 2.1
### Merge all outlet points together and calculate distance to coastline
Calculate Distance to Coast from outlet catchment point to the nearest coastline as a straight line distance
 * Generate near table and export as seperate csv

NHDPlus Section


In [12]:
import arcpy, datetime
import numpy as pd

# Input path to coastline
coast = r"D:\\Basedata\\AKSSF_Basedata\\AKSSF_Basedata.gdb\\NHD_H_Alaska_Coastline_alb"

# Merge all catchment outlet centroids together
nhdoutletsname = 'AKSSF_NHDPlus_awcHuc12_outlet_cats_points'
nhdoutletspath = os.path.join(outgdb, nhdoutletsname)

if not arcpy.Exists(nhdoutletspath):
    all_nhd_outlet_pts = arcpy.Merge_management(nhdplusoutlets,nhdoutletspath)
    # Start timing function
    start = datetime.datetime.now()
    print(f'Getting distance to coast {datetime.datetime.now()}...')
    arcpy.analysis.Near(all_nhd_outlet_pts, coast, None, "NO_LOCATION", "NO_ANGLE", "GEODESIC", "NEAR_DIST NEAR_DIST")
    arcpy.AlterField_management(all_nhd_outlet_pts,'NEAR_DIST','dist_catch_coast_km','dist_catch_coast_km' )
    arcpy.AddField_management(all_nhd_outlet_pts,'HUC12','TEXT')

    # Convert distance in meters to km
    with arcpy.da.UpdateCursor(all_nhd_outlet_pts,['dist_catch_coast_km','NHDPlusID','HUC12']) as cur:
        for row in cur:
            row[0] = row[0] * 0.001
            row[2] = nhdidDict[row[1]][1]
            cur.updateRow(row)
        del(row)
    del(cur)
    print(f'Process complete')
else:
    print(f'Outlet points already created at {nhdoutletspath}')


Outlet points already created at D:\GIS\AKSSF_awcHuc12_cv\AKSSF_awcHuc12_cv.gdb\AKSSF_NHDPlus_awcHuc12_outlet_cats_points


TauDEM Section


In [15]:
import arcpy, datetime
import numpy as pd

# Input path to coastline
coast = r"D:\\Basedata\\AKSSF_Basedata\\AKSSF_Basedata.gdb\\NHD_H_Alaska_Coastline_alb"

# Merge all catchment outlet centroids together
tauoutname = 'AKSSF_TauDEM_awcHuc12_outlet_cats_points'
tauoutpath = os.path.join(outgdb, tauoutname)

if not arcpy.Exists(tauoutpath):
    all_tau_outpts = arcpy.Merge_management(tauoutlets,tauoutpath)
    # Start timing function
    start = datetime.datetime.now()
    print(f'Getting distance to coast {datetime.datetime.now()}...')
    arcpy.analysis.Near(all_tau_outpts, coast, None, "NO_LOCATION", "NO_ANGLE", "GEODESIC", "NEAR_DIST NEAR_DIST")
    arcpy.AlterField_management(all_tau_outpts,'NEAR_DIST','dist_catch_coast_km','dist_catch_coast_km' )
    arcpy.AddField_management(all_tau_outpts,'HUC12','TEXT')
    arcpy.AddField_management(all_tau_outpts,'DSContAreaSqKM','DOUBLE')

    # Convert distance in meters to km
    with arcpy.da.UpdateCursor(all_tau_outpts,['dist_catch_coast_km','gridcode','HUC12','DSContArea','DSContAreaSqKM']) as cur:
        for row in cur:
            row[0] = row[0] * 0.001
            row[2] = tauidDict[row[1]][1]
            row[4] = int(row[3])/1000000 #convert sq meters to sq km
            cur.updateRow(row)
        del(row)
    del(cur)
    print(f'Process complete')
else:
    print(f'Outlet points already created at {tauoutpath}')

Getting distance to coast 2022-02-09 18:22:59.855894...
Process complete



## Merge NHD and Tau points together and export as CSV

In [16]:
# NHDPoints
nhdoutletsname = 'AKSSF_NHDPlus_awcHuc12_outlet_cats_points'
nhdoutletspath = os.path.join(outgdb, nhdoutletsname)

# Taupoints
tauoutname = 'AKSSF_TauDEM_awcHuc12_outlet_cats_points'
tauoutpath = os.path.join(outgdb, tauoutname)

# All points
catpointsname = 'AKSSF_awcHuc12_outlet_cats_points'
catpointspath = os.path.join(outgdb, catpointsname)

# Create FieldMappings object to manage merge output fields
out_fms = arcpy.FieldMappings()

# Add all fields from both point fcs
out_fms.addTable(nhdoutletspath)
out_fms.addTable(tauoutpath)

# Add input fields
out_fm_dsdrain = arcpy.FieldMap()
out_fm_dsdrain.addInputField(nhdoutletspath,'TotDASqKm')
out_fm_dsdrain.addInputField(tauoutpath,'DSContAreaSqKM')

# Set name of new output field "Street_Name"
dsdrain = out_fm_dsdrain.outputField
dsdrain.name = "DsContAreaSqKm"
out_fm_dsdrain.outputField = dsdrain

# add to field mappings
out_fms.addFieldMap(out_fm_dsdrain)

for field in out_fms.fields:
    if field.name not in ['cat_ID_con', 'DsContAreaSqKm','dist_catch_coast_km', 'HUC12']:
        out_fms.removeFieldMap(out_fms.findFieldMapIndex(field.name))

#if not arcpy.Exists(catpointspath):
addSourceInfo = "ADD_SOURCE_INFO"
cats_outlets = arcpy.Merge_management([nhdoutletspath,tauoutpath],
                                      catpointspath,
                                      out_fms,
                                      addSourceInfo)
print(f'Merging all available catchment outlet points')
#else:
#    print(f'AKSSF AWC Catchment outlets already identified and exported to {catpointspath}')


Merging all available catchment outlet points


### Convert to df and examine

In [17]:
import numpy
import pandas as pd

# Make catchment points df
cat_df = pd.DataFrame()
cat_field_list = []

for field in arcpy.ListFields(catpointspath):
    print(field.name)
    cat_field_list.append(field.name)
cat_arr = arcpy.da.TableToNumPyArray(catpointspath, ['cat_ID_con','dist_catch_coast_km','DsContAreaSqKm','HUC12'])
cat_df = pd.DataFrame(cat_arr)
cat_df = cat_df.set_index('cat_ID_con')
cat_df

OBJECTID
Shape
cat_ID_con
dist_catch_coast_km
HUC12
DsContAreaSqKm
MERGE_SRC


,dist_catch_coast_km,DsContAreaSqKm,HUC12
cat_ID_con,,,
Cook_Inlet_75004200000901,21.633384,36.96257508,190202020501
Cook_Inlet_75004200001724,17.599013,170.81217492,190202020503
Cook_Inlet_75004200001726,0.128861,570.1149751,190202020508
Cook_Inlet_75004200001493,0.147499,9.6406,190202020303
Cook_Inlet_75004200004105,0.137137,14.02302496,190202020102
...,...,...,...
Prince_William_Sound_91741,17.543384,572.85728,190202010906
Prince_William_Sound_91751,17.338805,574.793792,190202010905
Prince_William_Sound_91921,12.177571,739.347904,190202010907


### Export csv of outlet points for NHDPlus regions


In [18]:
import os
#Export CSV to read into R
catpts_outname = 'AKSSF_awcHuc12_dist_catch_coast_km.csv'
outlets_csv = os.path.join(outdir,catpts_outname)
if not arcpy.Exists(outlets_csv):
    arcpy.da.NumPyArrayToTable(cat_arr,outlets_csv)
    print('CSV export complete')
else:
    print(f'Csv of catchment outlet points already exported to {outlets_csv}')
print('----------')

CSV export complete
----------


## Section 3
### NHDPlus Watersheds
Generate Watersheds
* If watersheds have already been created there is no need to run this section again in order for subsequent process to run.

In [20]:
# NHDPLUS Watersheds

import arcpy, time, datetime, os
import pandas as pd
from functools import reduce
import arcpy, time, os, datetime, operator

arcpy.env.workspace = data_dir
regions = arcpy.ListWorkspaces()
arcpy.env.overwriteOutput = True
arcpy.env.qualifiedFieldNames = False
sr = arcpy.SpatialReference(3338)  #'NAD_1983_Alaska_Albers'
arcpy.env.outputCoordinateSystem = sr

wtdDict = {}

# Separate data by source type
nhdplus_dat = ['Cook_Inlet','Copper_River']
# Limit to Cook inlet for testing
# regions = ['D:\\GIS\\AKSSF\\Copper_River','D:\\GIS\\AKSSF\\Cook_Inlet']
regions = ['D:\\GIS\\AKSSF\\Prince_William_Sound']

# Start timing function
processStart = time.time()
processStartdt = datetime.datetime.now()

for region in regions:
    reg_start = time.time()
    roi = os.path.basename(region)
    print(roi)
    if roi in nhdplus_dat:
        try:
            wtdList = []
            arcpy.env.workspace = region
            gdb = arcpy.ListWorkspaces(workspace_type='FileGDB')
            ingdb = gdb[0]
            # set inputs
            vaa = os.path.join(ingdb, "vaa_merge")
            cats = os.path.join(ingdb, "cats_merge")
            streams = os.path.join(ingdb, "NHDFlowline_merge")
            outcats = os.path.join(ingdb, "awc_huc12_catchment_outlets")
            # Create list of nhdplus ids for outlet catchments
            idList = [int(row[0]) for row in arcpy.da.SearchCursor(outcats,'NHDPlusID')]
            #Make test list of few small catchments
            #idList = [75004400004166,75004400004344, 75004400010328]
            # Get list of index names for cats merge and add index if not already created
            index_names = [i.name for i in arcpy.ListIndexes(cats)]
            print(index_names)
            if 'NHDPlusID_index' not in index_names:
                print (f'Creating index for {cats}')
                arcpy.AddIndex_management(cats,'NHDPlusID','NHDPlusID_index')
            else:
                print(f'{cats} Indexed')

            #watersheds feature dataset for storing fcs
            fdatname = roi + '_Watersheds'
            fdat = os.path.join(outgdb,fdatname)
            if not arcpy.Exists(fdat):
                arcpy.management.CreateFeatureDataset(outgdb, fdatname, sr)
            else:
                print(f'{fdat} exists for {roi}')

            vaa_df1 = pd.DataFrame(arcpy.da.TableToNumPyArray(vaa, ("NHDPlusID", "FromNode", "ToNode", "StartFlag")))
            stream_df = pd.DataFrame(arcpy.da.TableToNumPyArray(streams, ("NHDPlusID", "FType")))
            dfs = [vaa_df1, stream_df]
            vaa_df = reduce(lambda left,right: pd.merge(left,right,on='NHDPlusID',how="outer"), dfs)
            # remove pipelines
            vaa_df = vaa_df[(vaa_df['FType'] != 428 )]
            vaa_df

            c=1
            for id in idList:
                iteration_start = time.time()
                print(f'{c}. Starting watershed for HUC {str(id)} ({(len(idList) - c)} remaining)')
                rec = [id]
                up_ids = []
                up_ids.append(rec)
                rec_len = len(rec)
                hws_sum = 0

                while rec_len != hws_sum:
                    fromnode = vaa_df.loc[vaa_df["NHDPlusID"].isin(rec), "FromNode"]
                    rec = vaa_df.loc[vaa_df["ToNode"].isin(fromnode), "NHDPlusID"]
                    rec_len = len(rec)
                    rec_hws = vaa_df.loc[vaa_df["ToNode"].isin(fromnode), "StartFlag"]
                    hws_sum = sum(rec_hws)
                    up_ids.append(rec)
                #up_ids is a list with more than numbers, use extend to only keep numeric nhdplusids
                newup_ids = []
                for x in up_ids:
                    newup_ids.extend(x)

                tempLayer = "catsLyr"
                expression = '"NHDPlusID" IN ({0})'.format(', '.join(map(str, newup_ids)) or 'NULL')
                arcpy.MakeFeatureLayer_management(cats, tempLayer, where_clause=expression)
                outdis = "memory/wtd_" + str(round(id))
                outwtd = os.path.join(fdat,f'{roi}_wtd_{str(int(id))}')
                dis = arcpy.Dissolve_management(tempLayer, outdis)
                watershed = arcpy.EliminatePolygonPart_management(dis, outwtd,"PERCENT", "0 SquareKilometers", 90, "CONTAINED_ONLY")
                wtdList.append(outwtd)
                append_value(wtdDict,roi,outwtd)

                # Stop iteration timer
                iteration_stop = time.time()
                iter_time = int (iteration_stop - iteration_start)
                print(f'Elapsed time: ({datetime.timedelta(seconds=iter_time)})')
                print(f'{"*"*60}')
                c+=1

            wtd_merge = arcpy.Merge_management(wtdList, os.path.join(ingdb,'awc_huc12_wtds_merge'),'','ADD_SOURCE_INFO')
            arcpy.AddField_management(wtd_merge,'cat_ID_con','TEXT')
            arcpy.AddField_management(wtd_merge,'cat_ID','DOUBLE')
            arcpy.AddField_management(wtd_merge,'cat_ID_txt','TEXT')
            arcpy.AddField_management(wtd_merge,'NHDPlusID','DOUBLE')
            with arcpy.da.UpdateCursor(wtd_merge,['MERGE_SRC','NHDPlusID','cat_ID_con','cat_ID','cat_ID_txt']) as cur:
                for row in cur:
                    # Pull nhdplus id from merge source and calculate fields
                    nhdplusid= int(row[0].split('_')[-1])
                    row[1] = nhdplusid
                    row[2] = roi + '_' + str(nhdplusid)
                    row[3] = nhdplusid
                    row[4] = str(nhdplusid)
                    cur.updateRow(row)
                del(row)
            del(cur)
            arcpy.CopyFeatures_management(wtd_merge,os.path.join(outgdb,f'{roi}_NhdAwcH12_wtds_merge' ))

            # Stop iteration timer
            reg_stop = time.time()
            reg_time = int (reg_stop - reg_start)
            print(f'{roi} Elapsed time: ({datetime.timedelta(seconds=reg_time)})')
            print(f'{"*"*100}')

        except:
            e = sys.exc_info()[1]
            print(e.args[0])
            arcpy.AddError(e.args[0])

    elif roi in tauDem_dat:
        try:
            reg_start = time.time()
            wtdList = []
            arcpy.env.workspace = region
            gdb = arcpy.ListWorkspaces(workspace_type='FileGDB')
            ingdb = gdb[0]
            # set inputs
            cats = os.path.join(ingdb, "cats_merge")
            streams = os.path.join(ingdb, "streams_merge")
            outcats = os.path.join(ingdb, "awc_huc12_catchment_outlets")
            # Create list of nhdplus ids for outlet catchments
            idList = [int(row[0]) for row in arcpy.da.SearchCursor(outcats,'gridcode')]
            index_names = [i.name for i in arcpy.ListIndexes(cats)]
            if 'catid_index' not in index_names:
                print (f'Creating index for {cats}')
                arcpy.AddIndex_management(cats, "catID", "catid_index")
            else:
                print(f'{cats} Indexed')
            #watersheds feature dataset for storing fcs
            fdatname = roi + '_Watersheds'
            fdat = os.path.join(outgdb,fdatname)
            if not arcpy.Exists(fdat):
                arcpy.management.CreateFeatureDataset(outgdb, fdatname, sr)
            else:
                print(f'{fdat} exists for {roi}')
            fields = arcpy.ListFields(streams)
            str_df = pd.DataFrame(arcpy.da.FeatureClassToNumPyArray(streams, ("LINKNO", "USLINKNO1", "USLINKNO2")))
            hws_codes = [-1]

            # Generate watersheds
            c=1
            for id in idList:
                iteration_start = time.time()
                print(f'{c}. Starting watershed for HUC {str(id)} ({(len(idList) - c)} remaining)')
                rec = [id]
                up_ids = []
                sum_rec = sum(rec)
                while(sum_rec > 0):
                    up_ids.append(rec)
                    rec = str_df.loc[str_df["LINKNO"].isin(rec), ("USLINKNO1", "USLINKNO2")]
                    rec = pd.concat([rec['USLINKNO1'], rec['USLINKNO2']])
                    sum_rec = sum(rec)
                # up_ids is a list with more than numbers, use extend to only keep numeric nhdplusids
                newup_ids = []
                for x in up_ids:
                    newup_ids.extend(x)

                tempLayer = "catsLyr"
                expression = '"gridcode" IN ({0})'.format(', '.join(map(str, newup_ids)) or 'NULL')
                arcpy.MakeFeatureLayer_management(cats, tempLayer)
                arcpy.management.SelectLayerByAttribute(tempLayer, "NEW_SELECTION", expression, None)
                print("Starting dissolve")
                outdis = "memory/wtd_" + str(round(id))
                outwtd = os.path.join(fdat,f'{roi}_wtd_{str(int(id))}')
                dis = arcpy.Dissolve_management(tempLayer, outdis)
                watershed = arcpy.EliminatePolygonPart_management(dis, outwtd,"PERCENT", "0 SquareKilometers", 90, "CONTAINED_ONLY")
                wtdList.append(watershed)
                append_value(wtdDict,roi,outwtd)

                # Stop iteration timer
                iteration_stop = time.time()
                iter_time = int (iteration_stop - iteration_start)
                print(f'Elapsed time: ({datetime.timedelta(seconds=iter_time)})')
                print(f'{"*"*60}')
                c+=1

            wtd_merge = arcpy.Merge_management(wtdList, os.path.join(ingdb,'awc_huc12_wtds_merge'),'','ADD_SOURCE_INFO')
            arcpy.AddField_management(wtd_merge,'cat_ID_con','TEXT')
            arcpy.AddField_management(wtd_merge,'cat_ID','DOUBLE')
            arcpy.AddField_management(wtd_merge,'cat_ID_txt','TEXT')
            with arcpy.da.UpdateCursor(wtd_merge,['gridcode','cat_ID_con','cat_ID','cat_ID_txt']) as cur:
                for row in cur:
                    row[1] = roi + '_' + str(row[0])
                    row[2] = int(row[0])
                    row[4] = str(row[0])
                    cur.updateRow(row)
                del(row)
            del(cur)
            arcpy.CopyFeatures_management(wtd_merge,os.path.join(outgdb, f'{roi}_NhdAwcH12_wtds_merge' ))

            # Stop iteration timer
            reg_stop = time.time()
            reg_time = int (reg_stop - reg_start)
            print(f'{roi} Elapsed time: ({datetime.timedelta(seconds=reg_time)})')
            print(f'{"*"*100}')

        except:
            e = sys.exc_info()[1]
            print(e.args[0])
            arcpy.AddError(e.args[0])
    else:
        print(f'{roi} not found - check inputs')
        sys.exit(f'{roi} not found - check inputs')

# End timing
processEnd = time.time()
processElapsed = int(processEnd - processStart)
processSuccess_time = datetime.datetime.now()

# Report success
print(f'Process completed at {processSuccess_time.strftime("%Y-%m-%d %H:%M")} '
      f'(Elapsed time: {datetime.timedelta(seconds=processElapsed)})')
print(f'{"*"*100}')




Prince_William_Sound
D:\GIS\AKSSF\Prince_William_Sound\Prince_William_Sound.gdb\cats_merge Indexed
OBJECTID
Shape
LINKNO
DSLINKNO
USLINKNO1
USLINKNO2
DSNODEID
strmOrder
Length
Magnitude
DSContArea
strmDrop
Slope
StraightL
USContArea
WSNO
DOUTEND
DOUTSTART
DOUTMID
Shape_Length
1. Starting watershed for HUC 3038 (111 remaining)
Starting dissolve
Elapsed time: (0:00:01)
************************************************************
2. Starting watershed for HUC 13227 (110 remaining)
Starting dissolve
Elapsed time: (0:00:01)
************************************************************
3. Starting watershed for HUC 17027 (109 remaining)
Starting dissolve
Elapsed time: (0:00:01)
************************************************************
4. Starting watershed for HUC 17697 (108 remaining)
Starting dissolve
Elapsed time: (0:00:01)
************************************************************
5. Starting watershed for HUC 18357 (107 remaining)
Starting dissolve
Elapsed time: (0:00:01)
**********

## TROUBLESHOOTING BLOCK
### Zonal statistics as table is failing with unknown error when run on watershed_merge and slope/elev rasters if using 'ALL' statistics.
Try alternative methods.  Below is test chunk for iterating over a list of stats individually and join results back to a copy of the merged watershed table.
* Cannot Run ZonalStatistics because tool does not process overlapping polygons as individual features whereas ZonalStatistics as table will




In [18]:
### TEST CHUNK###

import os, arcpy,time, datetime
from arcpy.sa import *
arcpy.env.overwriteOutput = True
#
# testoutgdb = r"C:\\Users\\dwmerrigan\\Documents\\GitHub\\ArcGIS_Default\\ArcGIS_Default.gdb"
# wtd_merge = r"D:\\GIS\\AKSSF_awcHuc12_cv\\AKSSF_awcHuc12_cv.gdb\\Cook_Inlet_NhdAwcH12_wtds_merge"
# wtd_cur_fields = ['cat_ID_txt', 'cat_ID',"cat_ID_con"]
# elev_rast = r"D:\\GIS\\AKSSF\\Cook_Inlet\\elev.tif"
# zstats = ['MIN_MAX_MEAN','STD']
# roi = 'Cook_Inlet'
#
# # Elevation variables
# wtd_merge_elev_table_name = roi + "_Watersheds_Merge_ElevZstats"
# wtd_merge_elev_table_path = os.path.join(testoutgdb, wtd_merge_elev_table_name)
#
# # list to store zonal stat tables
# wtdelevstats =[]
#
# # Create field mappings
# elev_fm = arcpy.FieldMap()
# elev_fms = arcpy.FieldMappings()
# for field in arcpy.ListFields(wtd_merge)[6:]:
#     elev_fm = arcpy.FieldMap()
#     elev_fm.addInputField(wtd_merge,field.name)
#     elev_fm.mergeRule = 'First'
#     # Set properties of the output name.
#     f_name = elev_fm.outputField
#     f_name.name = field.name
#     f_name.aliasName = field.name
#     elev_fm.outputField = f_name
#     elev_fms.addFieldMap(elev_fm)
#
# # Make copy of watershed merge input as table to join stats fields
# wtd_elev_metrics_table = arcpy.TableToTable_conversion(wtd_merge,
#                                                        testoutgdb,
#                                                        wtd_merge_elev_table_name,
#                                                        '',
#                                                        elev_fms,
#                                                        )
# # Add region identifier field for watershed tables                                                )
# arcpy.AddField_management(wtd_elev_metrics_table,'region',field_type='TEXT')
# # expression to calculate region field with roi name
# exp =  '"'+roi+'"'
# arcpy.CalculateField_management(wtd_elev_metrics_table,'region',exp)
#
# zstat_start = time.time()
# for stat in zstats:
#     outstattable = os.path.join(testoutgdb,f'{roi}_wtdElev{stat}')
#     zstat_start1 = time.time()
#     try:
#         print (f'running {stat}')
#         stat_table = ZonalStatisticsAsTable(in_zone_data = wtdmerge,
#                                             zone_field = wtd_cur_fields[0],
#                                             in_value_raster = elev_rast,
#                                             out_table = outstattable,
#                                             statistics_type=stat
#                                             )
#
#         stat_fields = [f.name for f in arcpy.ListFields(stat_table)]
#         arcpy.JoinField_management(wtd_elev_metrics_table,
#                                wtd_cur_fields[0],
#                                stat_table,
#                                wtd_cur_fields[0],
#                                stat_fields[5:]
#                                )
#
#         # Report time
#         zstat_stop1 = time.time()
#         zstat_time1 = int (zstat_stop1 - zstat_start1)
#         print(f'Watershed elev Zonal Stats for {stat} Elapsed time: ({datetime.timedelta(seconds=zstat_time1)})')
#         print(f'{"*"*100}')
#     except:
#         e = sys.exc_info()[1]
#         print(e.args[0])
#         arcpy.AddError(e.args[0])
# # Report time
# zstat_stop = time.time()
# zstat_time = int (zstat_stop - zstat_start)
# print(f'Watershed elev Zonal Stats for {roi} Elapsed time: ({datetime.timedelta(seconds=zstat_time)})')
# print(f'{"*"*100}')

## Section 4
Calculate Covariates


In [22]:
from arcpy.sa import *
arcpy.env.workspace = data_dir
arcpy.env.overwriteOutput = True
sr = arcpy.SpatialReference(3338) #'NAD_1983_Alaska_Albers'
arcpy.env.outputCoordinateSystem = sr
regions  = arcpy.ListWorkspaces(workspace_type="Folder")

# Lists for variables not needed at present time
#cat_asp_ztables = []
#wtd_asp_ztables = []
#cat_pernorth_taba_tables=[]

# Lists to store output tables
wtd_pernorth_taba_tables=[]
wtd_lp_tabint_tables = []
wtd_glac_tabint_tables = []
wtd_wet_taba_tables = []
cat_elev_ztables = []
wtd_elev_ztables = []
cat_slope_ztables = []
wtd_slope_ztables = []
lcld_Ztables = []

# Clear lists
cat_cur_fields = []
wtd_cur_fields = []

# Start timing function
processStart = time.time()
processStartdt = datetime.datetime.now()

# Split data by type
nhdplus_dat = ['Cook_Inlet','Copper_River']
tauDem_dat = ['Bristol_Bay', 'Kodiak', 'Prince_William_Sound']

#Limit to ci for testing
#regions = ['D:\\GIS\\AKSSF\\Cook_Inlet','D:\\GIS\\AKSSF\\Copper_River']
regions = ['D:\\GIS\\AKSSF\\Prince_William_Sound']

for region in regions:
    roi = os.path.basename(region)
    # expression to calculate region field with roi name
    exp =  '"'+roi+'"'
    if roi in nhdplus_dat:
        lakes_fc = nhd_lakes_fc
        # Fields for update cursor
        cat_cur_fields = ['cat_ID_txt', 'NHDPlusID',"cat_ID_con"]
        wtd_cur_fields = ['cat_ID_txt', 'cat_ID',"cat_ID_con"]
        print (f'{roi} in {nhdplus_dat} AKSSF list, using cat_fields {cat_cur_fields} and watershed fields {wtd_cur_fields}')
        print(f'{"*"*100}')
    # Set data and variables unique to regions with TauDEM Data
    elif roi in tauDem_dat:
        lakes_fc = tau_lakes_fc
        # Fields for update cursor
        if roi == 'Bristol_Bay':
            cat_cur_fields = ['cat_ID_txt', 'catID',"cat_ID_con"]
            wtd_cur_fields = ['cat_ID_txt', 'cat_ID',"cat_ID_con"]
        else:
            cat_cur_fields = ['cat_ID_txt', 'gridcode',"cat_ID_con"]
            wtd_cur_fields = ['cat_ID_txt', 'cat_ID',"cat_ID_con"]
        print (f'{roi} in {tauDem_dat} TauDEM list, using cat_fields {cat_cur_fields} and watershed fields {wtd_cur_fields}')
        print(f'{"*"*100}')
    # Start iter timing function
    iteration_start = time.time()
    # Set workspace to region folder
    arcpy.env.workspace = region
    walk = arcpy.da.Walk(region, datatype = ['FeatureClass','RasterDataset'])
    for dirpath, dirnames, filenames in walk:
        for filename in filenames:
            # Set merged watersheds dataset
            if 'awc_huc12_wtds_merge'== filename:
                wtdpath = os.path.join(dirpath,filename)
                wtdname = roi +'_'+ filename
                # Make local copy projected in AKAlbers
                wtd_merge = os.path.join(dirpath, filename)
                print(f'Merged watershed dataset {filename} found')
                print(f'{"*"*100}')
                wtdfieldnames = []
                wtdlstFields = arcpy.ListFields(wtd_merge)
                for field in wtdlstFields:
                    wtdfieldnames.append(field.name)
                if str(wtd_cur_fields[0]) in wtdfieldnames:
                    print (f'{wtd_cur_fields[0]} field already in dataset')
                    print(f'{"*"*100}')
                else:
                    print (f'Adding {wtd_cur_fields[0]} field to watershed dataset {wtd_merge}')
                    print(f'{"*"*100}')
                    # add cat_ID_txt field and concat cat_ID + region
                    arcpy.AddField_management(wtd_merge, str(wtd_cur_fields[0]),field_type='TEXT')
                    # populate cat_ID_txt
                    with arcpy.da.UpdateCursor(wtd_merge, wtd_cur_fields[0:2]) as cur:
                        for row in cur:
                            strval = str(row[1])
                            row[0] = strval.replace('.0',"")
                            # Update rows
                            cur.updateRow(row)
                        del(row)
                    del(cur)
                if str(wtd_cur_fields[2]) in wtdfieldnames:
                    print (f'{wtd_cur_fields[2]} field already in dataset {wtd_merge}')
                    print(f'{"*"*100}')
                else:
                    print (f'Adding {wtd_cur_fields[2]} field to watershed dataset {wtd_merge}')
                    print(f'{"*"*100}')
                    # add cat_ID_con field and concat cat_ID + region
                    arcpy.AddField_management(wtd_merge, str(wtd_cur_fields[2]),field_type='TEXT')
                    # populate cat_ID_txt
                    with arcpy.da.UpdateCursor(wtd_merge, wtd_cur_fields) as cur:
                        for row in cur:
                            strval = str(row[1])
                            row[2] = str(roi) +'_'+ strval.replace(".0","")
                            # Update rows
                            cur.updateRow(row)
                        del(row)
                    del(cur)

            # Select glaciers fc
            elif 'glaciers' == filename:
                # Make local copy projected in AKAlbers
                glacpath = os.path.join(dirpath, filename)
                glacname = roi+'_'+filename
                glac_fc = glacpath

            # Select elevation raster
            elif 'elev.tif' == filename:
                elev_rast = os.path.join(dirpath, filename)

            # # Select aspect raster
            # elif 'aspect' in filename:
            #     asp_rast = os.path.join(dirpath, filename)

            # Select north raster
            elif 'north.tif' == filename:
                nor_rast = os.path.join(dirpath, filename)

            # Select slope raster
            elif 'slope.tif' == filename:
                slope_rast = os.path.join(dirpath, filename)

            # Select wetland raster
            elif 'wetlands.tif' == filename:
                wet_rast = os.path.join(dirpath, filename)

            # Select catch_int fc (catchments of interest for region) and make a copy
            elif 'awc_huc12_catchment_outlets' == filename:
                # Make local copy projected in AKAlbers
                catspath = os.path.join(dirpath,filename)
                catsname = roi +"_"+filename
                cats = catspath
                catlstfields = arcpy.ListFields(cats)
                catfieldnames = []
                for field in catlstfields:
                    catfieldnames.append(field.name)
                if str(cat_cur_fields[0]) in catfieldnames:
                    print (f'{cat_cur_fields[0]} field already in dataset {cats}')
                    print(f'{"*"*100}')
                else:
                    print (f'Adding {cat_cur_fields[0]} field to catchment dataset {cats}')
                    print(f'{"*"*100}')
                    # add cat_ID_txt field
                    arcpy.AddField_management(cats, str(cat_cur_fields[0]), field_type='TEXT')
                    # populate cat_ID_txt
                    with arcpy.da.UpdateCursor(cats, cat_cur_fields[0:2]) as cur:
                        for row in cur:
                            strval = str(row[1])
                            row[0] = strval.replace('.0',"")
                            # Update rows
                            cur.updateRow(row)
                        del(row)
                    del(cur)
                if str(cat_cur_fields[2]) in catfieldnames:
                    print (f'{cat_cur_fields[2]} field already in dataset {cats}')
                    print(f'{"*"*100}')
                else:
                    print (f'Adding {cat_cur_fields[2]} field to catchment dataset {cats}')
                    print(f'{"*"*100}')
                    # add cat_ID_txt field & cat_ID + region concat field
                    arcpy.AddField_management(cats,str(cat_cur_fields[2]),field_type='TEXT')
                    # populate cat_ID_con
                    with arcpy.da.UpdateCursor(cats, cat_cur_fields) as cur:
                        for row in cur:
                            strval = str(row[1])
                            row[2] = str(roi) +'_'+ strval.replace('.0',"")
                            # Update rows
                            cur.updateRow(row)
                        del(row)
                    del(cur)

    print (f'Calculating topographic metrics for catchments & watersheds of interest in {roi} region')
    print ('----------')
    print(f'Geodatabase: {outgdb}')
    print ('----------')
    print (f'Watershed Merge: {wtd_merge}')
    print (f'  Projection {arcpy.Describe(wtd_merge).spatialReference.name}')
    print ('----------')
    print (f'HUC12 Catchment Outlets: {cats}')
    print (f'  Projection {arcpy.Describe(cats).spatialReference.name}')
    print ('----------')
    print (f'Elevation Raster: {elev_rast}')
    print (f'  Projection: {arcpy.Describe(elev_rast).spatialReference.name}')
    print ('----------')
    print (f'North Aspect Raster: {nor_rast}')
    print (f'  Projection: {arcpy.Describe(nor_rast).spatialReference.name}')
    print ('----------')
    print (f'Wetlands Raster: {wet_rast}')
    print (f'  Projection {arcpy.Describe(wet_rast).spatialReference.name}')
    print ('----------')
    print (f'Slope Raster: {slope_rast}')
    print (f'  Projection {arcpy.Describe(slope_rast).spatialReference.name}')
    print ('----------')
    print (f'Lakes Ponds fc: {lakes_fc}')
    print (f'  Projection {arcpy.Describe(lakes_fc).spatialReference.name}')
    print ('----------')
    print (f'Glaciers fc: {glac_fc} ')
    print (f'  Projection {arcpy.Describe(glac_fc).spatialReference.name}')
    print ('----------')
    print (f'{arcpy.GetCount_management(wtd_merge)} Watersheds to process')
    print ('----------')
    print (f'Catchment intersect {cats} selected')
    print ('----------')

    # # Aspect variables
    # wtd_merge_asp_table_name = roi + "_NhdAwcH12_wtd_mer_AspectZstats"
    # wtd_merge_asp_table_path = os.path.join(outgdb, wtd_merge_asp_table_name)
    # cat_asp_table_name = roi + "_NhdAwcH12_cats_AspectZstats"
    # cat_asp_table_path = os.path.join(outgdb, cat_asp_table_name)

    # Percent North variables
    wtd_merge_pernorth_table_name = roi + "_NhdAwcH12_wtd_mer_PerNorth"
    wtd_merge_pernorth_table_path = os.path.join(outgdb, wtd_merge_pernorth_table_name)
    # cat_pernorth_table_name = roi + "_NhdAwcH12_cats_PercentNorth"
    # cat_pernorth_table_path = os.path.join(outgdb, cat_pernorth_table_name)

    # Elevation variables
    wtd_merge_elev_table_name = roi + "_NhdAwcH12_wtd_mer_ElevZstats"
    wtd_merge_elev_table_path = os.path.join(outgdb, wtd_merge_elev_table_name)
    cat_elev_table_name = roi + "_NhdAwcH12_cats_ElevZstats"
    cat_elev_table_path = os.path.join(outgdb, cat_elev_table_name)

    # Slope variables
    wtd_merge_slope_table_name = roi + "_NhdAwcH12_wtd_mer_SlopeZstats"
    wtd_merge_slope_table_path = os.path.join(outgdb, wtd_merge_slope_table_name)
    cat_slope_table_name = roi + "_NhdAwcH12_cats_SlopeZstats"
    cat_slope_table_path = os.path.join(outgdb, cat_slope_table_name)

    # Lakes Ponds variables
    wtd_merge_lp_table_name = roi + "_NhdAwcH12_wtd_mer_PerLakes"
    wtd_merge_lp_table_path = os.path.join(outgdb, wtd_merge_lp_table_name)
    cat_lp_table_name = roi + "_NhdAwcH12_cats_PerLakes"
    cat_lp_path = os.path.join(outgdb, cat_lp_table_name)

    # Wetlands variables
    wtd_merge_wetlands_table_name = roi + "_NhdAwcH12_wtd_mer_PerWet"
    wtd_merge_wetlands_table_path = os.path.join(outgdb, wtd_merge_wetlands_table_name)
    cat_wetlands_table_name = roi + "NhdAwcH12_cats_PerWet"
    cat_wetlands_table_path = os.path.join(outgdb, cat_wetlands_table_name)

    # Glaciers
    wtd_merge_glac_table_name = roi + "_NhdAwcH12_wtd_mer_PerGlac"
    wtd_merge_glac_table_path = os.path.join(outgdb, wtd_merge_glac_table_name)
    cat_glac_table_name = roi + "_NhdAwcH12_cats_Glaciers"
    cat_glac_table_path = os.path.join(outgdb, cat_glac_table_name)

    try: # Zonal Stats section
        print(f'Begin Slope zonal statistics min/mean/max std dev for watersheds and catchments in {roi}'
              f' region')
        # Statistics to run for watersheds - 'ALL' is not an option at this time as tool will fail with unknown error
        zstats = ['MIN_MAX_MEAN','STD']
        # Begin Zonal Stats
        zstat_start = time.time()
        zstat_start1 = time.time()

        # Watershed slope Zonal Statistics
        print(f'Calculating {roi} watershed slope zonal stats...')
        arcpy.env.snapRaster = slope_rast
        arcpy.env.cellSize = slope_rast

        # Create field mappings
        slope_fm = arcpy.FieldMap()
        slope_fms = arcpy.FieldMappings()
        for field in arcpy.ListFields(wtd_merge)[6:]:
            slope_fm = arcpy.FieldMap()
            slope_fm.addInputField(wtd_merge,field.name)
            slope_fm.mergeRule = 'First'
            # Set properties of the output name.
            f_name = slope_fm.outputField
            f_name.name = field.name
            f_name.aliasName = field.name
            slope_fm.outputField = f_name
            slope_fms.addFieldMap(slope_fm)

        # Make copy of watershed merge input as table to join stats fields
        wtd_slope_metrics_table = arcpy.TableToTable_conversion(wtd_merge,
                                                               outgdb,
                                                               wtd_merge_slope_table_name,
                                                               '',
                                                               slope_fms,
                                                               )
        # Add region identifier field for watershed tables                                                )
        arcpy.AddField_management(wtd_slope_metrics_table,'region',field_type='TEXT')
        arcpy.CalculateField_management(wtd_slope_metrics_table,'region',exp)

        for stat in zstats:
            outstattable = os.path.join(outgdb,f'{roi}_wtdSlope_{stat}')
            zstat_start1 = time.time()
            print (f'running {stat}')
            stat_table = ZonalStatisticsAsTable(in_zone_data = wtd_merge,
                                                zone_field = wtd_cur_fields[0],
                                                in_value_raster = slope_rast,
                                                out_table = outstattable,
                                                statistics_type=stat
                                                )

            stat_fields = [f.name for f in arcpy.ListFields(stat_table)]
            arcpy.JoinField_management(wtd_slope_metrics_table,
                                   wtd_cur_fields[0],
                                   stat_table,
                                   wtd_cur_fields[0],
                                   stat_fields[5:] # Keep only stat field/s
                                   )

            # Report time
            zstat_stop1 = time.time()
            zstat_time1 = int (zstat_stop1 - zstat_start1)
            print(f'Watershed Slope Zonal {stat} for {roi} complete.\nElapsed time: ({datetime.timedelta(seconds=zstat_time1)})')
            print(f'{"*"*100}')

        # Append watershed slope table to list
        wtd_slope_ztables.append(wtd_slope_metrics_table)


        # Elevation Zonal statistics  for watersheds
        print(f'Begin Elevation zonal statistics min/mean/max std dev for watersheds and catchments in {roi}'
              f' region')
        zstat_start2 = time.time()
        arcpy.env.snapRaster = elev_rast
        arcpy.env.cellSize = elev_rast

        # Create field mappings
        elev_fm = arcpy.FieldMap()
        elev_fms = arcpy.FieldMappings()
        for field in arcpy.ListFields(wtd_merge)[6:]:
            elev_fm = arcpy.FieldMap()
            elev_fm.addInputField(wtd_merge,field.name)
            elev_fm.mergeRule = 'First'
            # Set properties of the output name.
            f_name = elev_fm.outputField
            f_name.name = field.name
            f_name.aliasName = field.name
            elev_fm.outputField = f_name
            elev_fms.addFieldMap(elev_fm)

        # Make copy of watershed merge input as table to join stats fields
        wtd_elev_metrics_table = arcpy.TableToTable_conversion(wtd_merge,
                                                               outgdb,
                                                               wtd_merge_elev_table_name,
                                                               '',
                                                               elev_fms,
                                                               )
        # Add region identifier field for watershed tables                                                )
        arcpy.AddField_management(wtd_elev_metrics_table,'region',field_type='TEXT')
        arcpy.CalculateField_management(wtd_elev_metrics_table,'region',exp)

        for stat in zstats:
            outstattable = os.path.join(outgdb,f'{roi}_wtdElev_{stat}')
            zstat_start1 = time.time()
            print (f'running {stat}')
            stat_table = ZonalStatisticsAsTable(in_zone_data = wtd_merge,
                                                zone_field = wtd_cur_fields[0],
                                                in_value_raster = elev_rast,
                                                out_table = outstattable,
                                                statistics_type=stat
                                                )

            stat_fields = [f.name for f in arcpy.ListFields(stat_table)]
            arcpy.JoinField_management(wtd_elev_metrics_table,
                                   wtd_cur_fields[0],
                                   stat_table,
                                   wtd_cur_fields[0],
                                   stat_fields[5:] # Keep only stat field/s
                                   )

            # Report time
            zstat_stop2 = time.time()
            zstat_time2 = int (zstat_stop2 - zstat_start2)
            print(f'Watershed Elevation Zonal {stat} for {roi} complete.\nElapsed time: ({datetime.timedelta(seconds=zstat_time2)})')
            print(f'{"*"*100}')
        # Append watershed elev table to list
        wtd_elev_ztables.append(wtd_elev_metrics_table)


        # Elevation zonal statistics for catchments
        print(f'Calculating {roi} catchment elevation zonal stats...')
        zstat_start3 = time.time()
        arcpy.env.snapRaster = elev_rast
        arcpy.env.cellSize = elev_rast
        cat_elev_metrics_table = ZonalStatisticsAsTable(in_zone_data = cats ,
                                                        zone_field = cat_cur_fields[0],
                                                        in_value_raster = elev_rast,
                                                        out_table = cat_elev_table_path,
                                                        statistics_type='ALL'
                                                        )
        # Add region identifier field for catchment table
        arcpy.AddField_management(cat_elev_metrics_table,'region',field_type='TEXT')
        # Add cat_ID_Con field
        arcpy.AddField_management(cat_elev_metrics_table,'cat_ID_con',field_type='TEXT')

        # Update fields
        with arcpy.da.UpdateCursor(cat_elev_metrics_table,['region','cat_ID_txt','cat_ID_con']) as cur:
            for row in cur:
                row[0] = roi
                strval = str(row[1])
                row[2] = roi+"_"+strval.replace(".0","")
                # Update
                cur.updateRow(row)
            del(row)
        del(cur)
        # Append catchment elev table to list
        cat_elev_ztables.append(cat_elev_metrics_table)
        # Report time
        zstat_stop3 = time.time()
        zstat_time3 = int (zstat_stop3 - zstat_start3)
        print(f'Elevation Zonal Stats for {roi} catchments complete.\nElapsed time: ({datetime.timedelta(seconds=zstat_time3)})')
        print(f'{"*"*100}')

        # Slope zonal statistics for catchments
        zstat_start4 = time.time()
        print(f'Calculating {roi} catchment slope zonal stats...')
        arcpy.env.snapRaster = slope_rast
        arcpy.env.cellSize = slope_rast
        cat_slope_metrics_table = ZonalStatisticsAsTable(in_zone_data = cats ,
                                                        zone_field = cat_cur_fields[0],
                                                        in_value_raster = slope_rast,
                                                        out_table = cat_slope_table_path,
                                                        statistics_type='ALL'
                                                        )
        # Add region identifier field for catchment table
        arcpy.AddField_management(cat_slope_metrics_table,'region',field_type='TEXT')
        # Add cat_ID_Con field
        arcpy.AddField_management(cat_slope_metrics_table,'cat_ID_con',field_type='TEXT')

        # Update region field
        with arcpy.da.UpdateCursor(cat_slope_metrics_table,['region','cat_ID_txt','cat_ID_con']) as cur:
            for row in cur:
                row[0] = roi
                strval =str(row[1])
                row[2] = roi+"_"+strval.replace(".0","")
                # Update
                cur.updateRow(row)
            del(row)
        del(cur)
        # Append catchment slope table to list
        cat_slope_ztables.append(cat_slope_metrics_table)
        # Report time
        zstat_stop4 = time.time()
        zstat_time4 = int (zstat_stop4 - zstat_start4)
        print(f'Slope Zonal Stats for {roi} catchments complete.\nElapsed time: ({datetime.timedelta(seconds=zstat_time4)})')
        print(f'{"*"*100}')


        # # Aspect Zonal statistics  for watersheds
        # print(f'Calculating {roi} watershed aspect zonal stats...')
        # wtd_asp_metrics_table = ZonalStatisticsAsTable(in_zone_data = wtd_merge, zone_field ="cat_ID_txt",
        #                                                in_value_raster = asp_rast, out_table = wtd_merge_asp_table_path,
        #                                                statistics_type='ALL')
        # arcpy.AddField_management(wtd_asp_metrics_table, 'region', field_type='TEXT')
        # Add cat_ID_Con field
        # arcpy.AddField_management(wtd_asp_metrics_table,'cat_ID_con',field_type='TEXT')
        # arcpy.CalculateField_management(wtd_asp_metrics_table, 'region', 'roi')
        # Update region field
        # with arcpy.da.UpdateCursor(wtd_asp_metrics_table,['region','cat_ID_txt','cat_ID_con']) as cur:
        #     for row in cur:
        #         row[0] = roi
        #         strval = str(row[1])
        #         row[2] = roi+"_"+strval.replace(".0","")
        #         # Update
        #         cur.updateRow(row)
        #     del(row)
        # del(cur)
        # wtd_asp_ztables.append(wtd_asp_metrics_table)

        # # Aspect Zonal statistics for catchments
        # print(f'Calculating {roi} catchment aspect zonal stats...')
        # cat_asp_metrics_table = ZonalStatisticsAsTable(in_zone_data = cats, zone_field ="cat_ID_txt",
        #                                                in_value_raster = asp_rast, out_table = cat_asp_table_path,
        #                                                statistics_type='ALL')
        # arcpy.AddField_management(cat_asp_metrics_table, 'region', field_type='TEXT')
        # Add cat_ID_Con field
        # arcpy.AddField_management(cat_asp_metrics_table,'cat_ID_con',field_type='TEXT')
        # arcpy.CalculateField_management(cat_asp_metrics_table, 'region', 'roi')
        # Update region field
        # with arcpy.da.UpdateCursor(cat_asp_metrics_table,['region','cat_ID_txt','cat_ID_con']) as cur:
        #     for row in cur:
        #         strval = str(row[1])
        #         row[2] = roi+"_"+strval.replace(".0","")
        #         # Update
        #         cur.updateRow(row)
        #     del(row)
        # del(cur)
        # cat_asp_ztables.append(cat_asp_metrics_table)

        zstat_stop = time.time()
        zstat_time = int (zstat_stop - zstat_start)
        print(f'All Zonal Stats for {roi} Elapsed time: ({datetime.timedelta(seconds=zstat_time)})')
        print(f'{"*"*100}')

        # Tabulate Area with north grid and watersheds
        tabarea_start = time.time()
        tabarea_start1 = time.time()
        print(f'Begin tabulate area of north facing cells for watersheds and catchments in {roi} region')
        print(f'{"*"*100}')
        # Percent North Tabulate area for watersheds
        wtd_per_north_tabarea = arcpy.sa.TabulateArea(in_zone_data= wtd_merge,
                                                      zone_field= wtd_cur_fields[0],
                                                      in_class_data=nor_rast,
                                                      class_field="Value",
                                                      out_table = wtd_merge_pernorth_table_path
                                                      )
        # Add region and percent north fields
        arcpy.AlterField_management(wtd_per_north_tabarea,'CAT_ID_TXT','CAT_ID_TXT_DEL','CAT_ID_TXT_DEL')
        arcpy.AddField_management(wtd_per_north_tabarea, 'region', field_type='TEXT')
        arcpy.AddField_management(wtd_per_north_tabarea, 'NhdAwcH12_wtd_north_per', field_type='Float')
        arcpy.AddField_management(wtd_per_north_tabarea, wtd_cur_fields[0], field_type='TEXT')
        arcpy.AddField_management(wtd_per_north_tabarea, wtd_cur_fields[2], field_type='TEXT')
        wtdnorfields = [f.name for f in arcpy.ListFields(wtd_per_north_tabarea)]
        #print (wtdnorfields)
        with arcpy.da.UpdateCursor(wtd_per_north_tabarea, wtdnorfields) as cur:
            for row in cur:
                strval = str(row[1])
                row[4] = roi
                row[5] = row[3]/(row[3]+row[2])*100
                row[6] = strval.replace('.0','')
                row[7] = roi +'_'+ strval.replace(".0","")
                # Update
                cur.updateRow(row)
            del(row)
        del(cur)
        # Drop UPPERCASE field form tab area
        arcpy.DeleteField_management(wtd_per_north_tabarea,'CAT_ID_TXT_DEL')
        # Append watershed percent north table to list
        wtd_pernorth_taba_tables.append(wtd_per_north_tabarea)
        # Report tab area times
        tabarea_stop1 = time.time()
        tabarea_time1 = int (tabarea_stop1 - tabarea_start1)
        print(f'Watershed percent north Tabulate area/intersections for {roi} complete.\nElapsed time: ({datetime.timedelta(seconds=tabarea_time1)})')
        print(f'{"*"*100}')

        # Percent Lakes Ponds using Tabulate Intersection for watersheds
        print(f'Begin watershed percent lakes ponds for {roi}')
        tabarea_start2 = time.time()
        wtd_lp_tabint = arcpy.TabulateIntersection_analysis(wtd_merge,
                                                            zone_fields=wtd_cur_fields[0],
                                                            in_class_features=lakes_fc,
                                                            out_table=wtd_merge_lp_table_path,
                                                            class_fields='Ftype',
                                                            out_units="SQUARE_METERS"
                                                            )
        # Add region and cat id fields
        arcpy.AlterField_management(wtd_lp_tabint,'PERCENTAGE','NhdAwcH12_wtd_lake_per','NhdAwcH12_wtd_lake_per')
        arcpy.AlterField_management(wtd_lp_tabint,'AREA','NhdAwcH12_wtd_lake_area_sqm','NhdAwcH12_wtd_lake_area_sqm')
        arcpy.AddField_management(wtd_lp_tabint, 'region', field_type='TEXT')
        arcpy.AddField_management(wtd_lp_tabint, wtd_cur_fields[1], field_type='TEXT')
        arcpy.AddField_management(wtd_lp_tabint, wtd_cur_fields[2], field_type='TEXT')
        wtdlpfields = [f.name for f in arcpy.ListFields(wtd_lp_tabint)]
        #print (wtdlpfields)
        with arcpy.da.UpdateCursor(wtd_lp_tabint, wtdlpfields) as cur:
            for row in cur:
                strval = str(row[1])
                row[5] = roi
                row[6] = strval.replace('.0','')
                row[7] = roi +'_'+ strval.replace(".0","")
                # Update
                cur.updateRow(row)
            del(row)
        del(cur)

        # Append watershed lakes ponds table to list
        wtd_lp_tabint_tables.append(wtd_lp_tabint)
        # Report tab area times
        tabarea_stop2 = time.time()
        tabarea_time2 = int (tabarea_stop2 - tabarea_start2)
        print(f'Percent Lakes Tabulate area/intersections for {roi} complete.\nElapsed time: ({datetime.timedelta(seconds=tabarea_time2)})')
        print(f'{"*"*100}')

        # Percent glaciers using Tabulate Intersection for watersheds
        tabarea_start3 = time.time()
        print(f'Begin tabulate intersection between {glac_fc} and watersheds in {roi} region')
        print(f'{"*"*100}')
        wtd_glac_tabint = arcpy.TabulateIntersection_analysis(wtd_merge,
                                                            zone_fields=wtd_cur_fields[0],
                                                            in_class_features=glac_fc,
                                                            out_table=wtd_merge_glac_table_path,
                                                            class_fields='O1Region',
                                                            out_units="SQUARE_METERS"
                                                            )
        # Add region and cat id fields
        arcpy.AlterField_management(wtd_glac_tabint,'PERCENTAGE','NhdAwcH12_wtd_glacier_per','NhdAwcH12_wtd_glacier_per')
        arcpy.AlterField_management(wtd_glac_tabint,'AREA','NhdAwcH12_wtd_glacier_area_sqm','NhdAwcH12_wtd_glacier_area_sqm')
        arcpy.AddField_management(wtd_glac_tabint, 'region', field_type='TEXT')
        arcpy.AddField_management(wtd_glac_tabint, wtd_cur_fields[1], field_type='TEXT')
        arcpy.AddField_management(wtd_glac_tabint, wtd_cur_fields[2], field_type='TEXT')
        wtdglacfields = [f.name for f in arcpy.ListFields(wtd_glac_tabint)]
        #print (wtdglacfields)
        with arcpy.da.UpdateCursor(wtd_glac_tabint, wtdglacfields) as cur:
            for row in cur:
                strval = str(row[1])
                row[5] = roi
                row[6] = strval.replace('.0','')
                row[7] = roi +'_'+ strval.replace(".0","")
                # Update
                cur.updateRow(row)
            del(row)
        del(cur)
        # Append watershed percent glacier table to list
        wtd_glac_tabint_tables.append(wtd_glac_tabint)
        # Report tab area times
        tabarea_stop3 = time.time()
        tabarea_time3 = int (tabarea_stop3 - tabarea_start3)
        print(f'Percent Glacier Tabulate area/intersections for {roi} complete.\nElapsed time: ({datetime.timedelta(seconds=tabarea_time3)})')
        print(f'{"*"*100}')

        # Tabulate Area with wetlands grid and watersheds
        tabarea_start4 = time.time()
        print(f'Begin tabulate intersection between {wet_rast} and watersheds in {roi} region')
        print(f'{"*"*100}')
        # Wetlands tabulate area for watersheds
        wtd_per_wet_tabarea = arcpy.sa.TabulateArea(in_zone_data= wtd_merge,
                                                      zone_field= wtd_cur_fields[0],
                                                      in_class_data=wet_rast,
                                                      class_field="Value",
                                                      out_table=wtd_merge_wetlands_table_path
                                                      )
        # Add region and percent wet fields
        arcpy.AlterField_management(wtd_per_wet_tabarea,'CAT_ID_TXT','CAT_ID_TXT_DEL','CAT_ID_TXT_DEL')
        arcpy.AddField_management(wtd_per_wet_tabarea, 'region', field_type='TEXT')
        arcpy.AddField_management(wtd_per_wet_tabarea, 'NhdAwcH12_wtd_wet_per', field_type='Float')
        arcpy.AddField_management(wtd_per_wet_tabarea, wtd_cur_fields[0], field_type='TEXT')
        arcpy.AddField_management(wtd_per_wet_tabarea, wtd_cur_fields[2], field_type='TEXT')
        wtdwetfields = [f.name for f in arcpy.ListFields(wtd_per_wet_tabarea)]
        #print (wtdwetfields)
        with arcpy.da.UpdateCursor(wtd_per_wet_tabarea, wtdwetfields) as cur:
            for row in cur:
                strval = str(row[1])
                row[4] = roi
                row[5] = row[3]/(row[3]+row[2])*100
                row[6] = strval.replace('.0','')
                row[7] = roi +'_'+ strval.replace(".0","")
                # Update
                cur.updateRow(row)
            del(row)
        del(cur)
        # Drop UPPERCASE field form tab area
        arcpy.DeleteField_management(wtd_per_wet_tabarea,'CAT_ID_TXT_DEL')
        # Append watershed percent wetlands table to list
        wtd_wet_taba_tables.append(wtd_per_wet_tabarea)
        # Report tab area times
        tabarea_stop4 = time.time()
        tabarea_time4 = int (tabarea_stop4 - tabarea_start4)
        print(f'Percent Wetlands Tabulate area/intersections for {roi} complete.\nElapsed time: ({datetime.timedelta(seconds=tabarea_time4)})')
        print(f'{"*"*100}')

        # # Percent North Tabulate Area for catchments
        # cat_per_north_tabarea = arcpy.sa.TabulateArea(in_zone_data= cats, zone_field='cat_ID_con',
        #                                             in_class_data=nor_rast,"Value",
        #                                             out_table=cat_pernorth_table_path)

        # # Add and calculate region identifier field for catchment table
        # arcpy.AlterField_management(cat_per_north_tabarea,'CAT_ID_TXT','CAT_ID_TXT_DEL','CAT_ID_TXT_DEL')
        # arcpy.AddField_management(cat_per_north_tabarea, 'region', field_type='TEXT')
        # arcpy.AddField_management(cat_per_north_tabarea, 'cat_north_per', field_type='Float')
        # arcpy.AddField_management(cat_per_north_tabarea, cat_cur_fields[0], field_type='TEXT')
        # arcpy.AddField_management(cat_per_north_tabarea, cat_cur_fields[2], field_type='TEXT')
        # catnorfields = [f.name for f in arcpy.ListFields(cat_per_north_tabarea)]
        # print (catnorfields)
        # with arcpy.da.UpdateCursor(cat_per_north_tabarea,catnorfields) as cur:
        #     for row in cur:
        #         strval = str(row[1])
        #         row[4] = roi
        #         row[5] = row[3]/(row[3]+row[2])*100
        #         row[6] = strval.replace('.0','')
        #         row[7] = roi +'_'+ strval.replace(".0","")
        #         # Update
        #         cur.updateRow(row)
        #     del(row)
        # del(cur)
        # Drop UPPERCASE field form tab area
        # arcpy.DeleteField_management(cat_per_north_tabarea,'CAT_ID_TXT_DEL')
        # # Append catchment percent north table to list
        # cat_pernorth_taba_tables.append(cat_per_north_tabarea)
        # Report tab area times
        tabarea_stop = time.time()
        tabarea_time = int (tabarea_stop - tabarea_start)
        print(f'Tabulate area/intersections for {roi} complete\nElapsed time: ({datetime.timedelta(seconds=tabarea_time)})')
        print(f'{"*"*100}')

        # Begin LCLD calculations
        walk = arcpy.da.Walk(lcld_folder, datatype='RasterDataset')
        for dirpath, dirnames, filenames in walk:
            for filename in filenames:
                raspath = os.path.join(dirpath, filename)
                year = filename[0:4]
                lcld_outname = roi+'NhdAwcH12_lcld_'+str(year)+'_zStats'
                lcld_outpath = os.path.join(outgdb, lcld_outname)
                print(f'Year: {year} - raster path {raspath}')
                colname = 'NhdAwcH12_wtd_lcld_mn_' + str(year)
                # lcld zonal statistics as table for all akssf watersheds
                print(f'Calculating {filename} zonal stats for all {roi} watersheds...')
                #arcpy.env.snapRaster = raspath
                #arcpy.env.cellSize = raspath

                # Begin Zonal Stats
                lcldzstat_start = time.time()
                print(f'Begin zonal stats for {filename}')
                lcld_table = ZonalStatisticsAsTable(in_zone_data = wtd_merge,
                                                                zone_field = 'cat_ID_con',
                                                                in_value_raster = raspath,
                                                                out_table = lcld_outpath,
                                                                statistics_type='MEAN'
                                                                )
                # Append zTable to table list
                lcld_Ztables.append(lcld_outpath)
                arcpy.AlterField_management(lcld_table,'MEAN', colname,colname)
                proc_list = [row[0] for row in arcpy.da.SearchCursor(lcld_table,'cat_ID_con')]
                lcldzstat_stop = time.time()
                lcldzstat_time = int (lcldzstat_stop - lcldzstat_start)
                print(f'Zonal Stats for {filename} - Elapsed time: ({datetime.timedelta(seconds=lcldzstat_time)})')


    except:
        e = sys.exc_info()[1]
        print(f'ERRFLAG!!! = {e.args[0]}')
        arcpy.AddError(e.args[0])

    iter_stop = time.time()
    iter_time = int(iter_stop - iteration_start)
    print(f'All Covariates for {roi} completed.\nElapsed time: ({datetime.timedelta(seconds=iter_time)})')
    print(f'{"*"*100}')

# End timing
processEnd = time.time()
processElapsed = int(processEnd - processStart)
processSuccess_time = datetime.datetime.now()

# Report success
print(f'Process completed at {processSuccess_time.strftime("%Y-%m-%d %H:%M")} '
      f'(Elapsed time: {datetime.timedelta(seconds=processElapsed)})')
print(f'{"*"*100}')



Prince_William_Sound in ['Bristol_Bay', 'Kodiak', 'Prince_William_Sound'] TauDEM list, using cat_fields ['cat_ID_txt', 'gridcode', 'cat_ID_con'] and watershed fields ['cat_ID_txt', 'cat_ID', 'cat_ID_con']
****************************************************************************************************
Adding cat_ID_txt field to catchment dataset D:\GIS\AKSSF\Prince_William_Sound\Prince_William_Sound.gdb\awc_huc12_catchment_outlets
****************************************************************************************************
Adding cat_ID_con field to catchment dataset D:\GIS\AKSSF\Prince_William_Sound\Prince_William_Sound.gdb\awc_huc12_catchment_outlets
****************************************************************************************************
Merged watershed dataset awc_huc12_wtds_merge found
****************************************************************************************************
cat_ID_txt field already in dataset
***************************************

Examine LCLD tables and merge/export


In [22]:
dfs = []
for table in lcld_Ztables:
    tblname = table[-16:]
    print(tblname)
    dfname = tblname + '_arr'
    # Make df
    dfname = pd.DataFrame()
    lcld_field_list = []
    for field in arcpy.ListFields(table):
        lcld_field_list.append(field.name)
        #print(f'{field.name}')
    lcld_arr = arcpy.da.TableToNumPyArray(table, lcld_field_list)
    dfname = pd.DataFrame(lcld_arr)
    dfname = dfname.drop(['OBJECTID','ZONE_CODE', 'AREA', 'COUNT'],axis=1)
    dfname = dfname.set_index('cat_ID_con')
    dfs.append(dfname)

# Merge all data frames together
import numpy as np
from functools import reduce
lcld_df = reduce(lambda left,right: pd.merge(left,right,on='cat_ID_con',how="outer"), dfs)
lcld_df

lcld_2001_zStats
lcld_2002_zStats
lcld_2003_zStats
lcld_2004_zStats
lcld_2005_zStats
lcld_2006_zStats
lcld_2007_zStats
lcld_2008_zStats
lcld_2009_zStats
lcld_2010_zStats
lcld_2011_zStats
lcld_2012_zStats
lcld_2013_zStats
lcld_2014_zStats
lcld_2015_zStats
lcld_2016_zStats
lcld_2017_zStats
lcld_2018_zStats
lcld_2019_zStats
lcld_2001_zStats
lcld_2002_zStats
lcld_2003_zStats
lcld_2004_zStats
lcld_2005_zStats
lcld_2006_zStats
lcld_2007_zStats
lcld_2008_zStats
lcld_2009_zStats
lcld_2010_zStats
lcld_2011_zStats
lcld_2012_zStats
lcld_2013_zStats
lcld_2014_zStats
lcld_2015_zStats
lcld_2016_zStats
lcld_2017_zStats
lcld_2018_zStats
lcld_2019_zStats


,NhdAwcH12_wtd_lcld_mn_2001_x,NhdAwcH12_wtd_lcld_mn_2002_x,NhdAwcH12_wtd_lcld_mn_2003_x,NhdAwcH12_wtd_lcld_mn_2004_x,NhdAwcH12_wtd_lcld_mn_2005_x,NhdAwcH12_wtd_lcld_mn_2006_x,NhdAwcH12_wtd_lcld_mn_2007_x,NhdAwcH12_wtd_lcld_mn_2008_x,NhdAwcH12_wtd_lcld_mn_2009_x,NhdAwcH12_wtd_lcld_mn_2010_x,...,NhdAwcH12_wtd_lcld_mn_2010_y,NhdAwcH12_wtd_lcld_mn_2011_y,NhdAwcH12_wtd_lcld_mn_2012_y,NhdAwcH12_wtd_lcld_mn_2013_y,NhdAwcH12_wtd_lcld_mn_2014_y,NhdAwcH12_wtd_lcld_mn_2015_y,NhdAwcH12_wtd_lcld_mn_2016_y,NhdAwcH12_wtd_lcld_mn_2017_y,NhdAwcH12_wtd_lcld_mn_2018_y,NhdAwcH12_wtd_lcld_mn_2019_y
cat_ID_con,,,,,,,,,,,,,,,,,,,,,
Cook_Inlet_75004200000901,529.096139,522.720965,522.882768,514.113400,520.218164,533.944318,510.872831,535.508999,513.403234,529.247769,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Cook_Inlet_75004200001724,532.838425,526.509814,516.782917,518.399955,516.323943,528.420731,519.110808,537.231471,514.988760,528.820847,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Cook_Inlet_75004200001726,532.696767,528.730766,518.955074,522.615145,519.107225,529.118583,524.365134,542.061418,518.244161,530.271466,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Cook_Inlet_75004200001493,537.966327,532.807992,506.473526,522.566349,504.502532,518.689099,526.816053,553.797604,519.326562,542.378073,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Cook_Inlet_75004200004105,552.605999,547.518107,520.284118,527.237473,515.436355,531.036268,534.917731,561.502107,536.340433,550.691235,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
Copper_River_75003900029086,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,472.782352,458.794501,485.641902,487.820103,423.330805,414.759543,410.158446,470.742159,451.558074,439.999561
Copper_River_75003900044552,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,505.846055,494.667803,529.897337,514.894985,479.218491,414.404403,425.842085,496.148778,491.614218,470.632806
Copper_River_75003900054944,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,475.444300,476.375077,505.542194,493.379799,470.743883,423.433889,413.119983,478.510037,472.765282,434.095321


In [23]:
# Export merged dataframe to csv
lcld_csv_out = os.path.join(outdir,'AKSSF_NHDPlus_AWC_HUC12_wtd_lcld_mn.csv')
lcld_df.to_csv(lcld_csv_out, encoding = 'utf-8')


## Drop unnecessary fields and rename as needed from merged tables.
- Create Key value dictionary and use update cursor to rename fields.

In [ ]:
# Table names/paths
wtd_per_north_table_out = os.path.join(outgdb, 'AKSSF_NHDPlus_awc_huc12_wtd_north_per')
cat_elev_table_out = os.path.join(outgdb,'AKSSF_NHDPlus_awc_huc12_cat_elev')
cat_slope_table_out = os.path.join(outgdb,'AKSSF_NHDPlus_awc_huc12_cat_slope')
wtd_elev_table_out = os.path.join(outgdb, 'AKSSF_NHDPlus_awc_huc12_wtd_elev')
wtd_per_glac_table_out = os.path.join(outgdb, 'AKSSF_NHDPlus_awc_huc12_wtd_glacier_per')
wtd_per_lp_table_out = os.path.join(outgdb, 'AKSSF_NHDPlus_awc_huc12_wtd_lakepond_per')
wtd_slope_table_out = os.path.join(outgdb, 'AKSSF_NHDPlus_awc_huc12_wtd_slope')
wtd_wet_table_out = os.path.join(outgdb, 'AKSSF_NHDPlus_awc_huc12_wtd_wetland_per')

# Merge all regional tables together
outtables = []
wtd_per_north = arcpy.Merge_management(wtd_pernorth_taba_tables, wtd_per_north_table_out)
arcpy.AlterField_management(wtd_per_north,"VALUE_0","NhdAwcH12_non_north_area","NhdAwcH12_non_north_area")
arcpy.AlterField_management(wtd_per_north,"VALUE_1","NhdAwcH12_north_area","NhdAwcH12_north_area")
outtables.append(wtd_per_north)
cat_elev = arcpy.Merge_management(cat_elev_ztables, cat_elev_table_out)
outtables.append(cat_elev)
wtd_elev = arcpy.Merge_management(wtd_elev_ztables, wtd_elev_table_out)
outtables.append(wtd_elev)
wtd_slope = arcpy.Merge_management(wtd_slope_ztables, wtd_slope_table_out)
outtables.append(wtd_slope)
cat_slope = arcpy.Merge_management(cat_slope_ztables, cat_slope_table_out)
outtables.append(cat_slope)
wtd_wet = arcpy.Merge_management(wtd_wet_taba_tables, wtd_wet_table_out)
arcpy.AlterField_management(wtd_wet,"VALUE_0","NhdAwcH12_non_wetland_area","NhdAwcH12_non_wetland_area")
arcpy.AlterField_management(wtd_wet,"VALUE_1","NhdAwcH12_wetland_area","NhdAwcH12_wetland_area")
outtables.append(wtd_wet)
wtd_glac = arcpy.Merge_management(wtd_glac_tabint_tables, wtd_per_glac_table_out)
outtables.append(wtd_glac)
wtd_lp = arcpy.Merge_management(wtd_lp_tabint_tables, wtd_per_lp_table_out)
outtables.append(wtd_lp)
print ('Tables merged')
print('----------')


In [ ]:
#Set up field dictionary
elevDict = { 'ZONE_CODE': ('NhdAwcH12_cat_elev_ZONE_CODE', 'NhdAwcH12_wtd_elev_ZONE_CODE'),
         'COUNT': ('NhdAwcH12_cat_elev_COUNT', 'NhdAwcH12_wtd_elev_COUNT'),
          'AREA': ('NhdAwcH12_cat_elev_AREA', 'NhdAwcH12_wtd_elev_AREA'),
          'MIN': ('NhdAwcH12_cat_elev_MIN', 'NhdAwcH12_wtd_elev_MIN'),
          'MAX': ('NhdAwcH12_cat_elev_MAX', 'NhdAwcH12_wtd_elev_MAX'),
          'RANGE': ('NhdAwcH12_cat_elev_RANGE', 'NhdAwcH12_wtd_elev_RANGE'),
          'MEAN': ('NhdAwcH12_cat_elev_MEAN', 'NhdAwcH12_wtd_elev_MEAN'),
          'STD': ('NhdAwcH12_cat_elev_STD', 'NhdAwcH12_wtd_elev_STD'),
          'SUM': ('NhdAwcH12_cat_elev_SUM', 'NhdAwcH12_wtd_elev_SUM'),
          'VARIETY': ('NhdAwcH12_cat_elev_VARIETY', 'NhdAwcH12_wtd_elev_VARIETY'),
          'MAJORITY': ('NhdAwcH12_cat_elev_MAJORITY', 'NhdAwcH12_wtd_elev_MAJORITY'),
          'MINORITY': ('NhdAwcH12_cat_elev_MINORITY', 'NhdAwcH12_wtd_elev_MINORITY'),
          'MEDIAN': ('NhdAwcH12_cat_elev_MEDIAN', 'NhdAwcH12_wtd_elev_MEDIAN'),
          'PCT90': ('NhdAwcH12_cat_elev_PCT90', 'NhdAwcH12_wtd_elev_PCT90')
         }

slopeDict = { 'ZONE_CODE': ('NhdAwcH12_cat_slope_ZONE_CODE', 'NhdAwcH12_wtd_slope_ZONE_CODE'),
         'COUNT': ('NhdAwcH12_cat_slope_COUNT', 'NhdAwcH12_wtd_slope_COUNT'),
          'AREA': ('NhdAwcH12_cat_slope_AREA', 'NhdAwcH12_wtd_slope_AREA'),
          'MIN': ('NhdAwcH12_cat_slope_MIN', 'NhdAwcH12_wtd_slope_MIN'),
          'MAX': ('NhdAwcH12_cat_slope_MAX', 'NhdAwcH12_wtd_slope_MAX'),
          'RANGE': ('NhdAwcH12_cat_slope_RANGE', 'NhdAwcH12_wtd_slope_RANGE'),
          'MEAN': ('NhdAwcH12_cat_slope_MEAN', 'NhdAwcH12_wtd_slope_MEAN'),
          'STD': ('NhdAwcH12_cat_slope_STD', 'NhdAwcH12_wtd_slope_STD'),
          'SUM': ('NhdAwcH12_cat_slope_SUM', 'NhdAwcH12_wtd_slope_SUM'),
          'VARIETY': ('NhdAwcH12_cat_slope_VARIETY', 'NhdAwcH12_wtd_slope_VARIETY'),
          'MAJORITY': ('NhdAwcH12_cat_slope_MAJORITY', 'NhdAwcH12_wtd_slope_MAJORITY'),
          'MINORITY': ('NhdAwcH12_cat_slope_MINORITY', 'NhdAwcH12_wtd_slope_MINORITY'),
          'MEDIAN': ('NhdAwcH12_cat_slope_MEDIAN', 'NhdAwcH12_wtd_slope_MEDIAN'),
          'PCT90': ('NhdAwcH12_cat_slope_PCT90', 'NhdAwcH12_wtd_slope_PCT90')
         }

# Rename fields for elevation tables
for field in arcpy.ListFields(wtd_elev):
    keyval = field.name
    if keyval in elevDict:
        newname = elevDict[keyval][1]
        newalias = elevDict[keyval][1]
        print (keyval, newname)
        arcpy.AlterField_management(wtd_elev, keyval, newname, newalias)

for field in arcpy.ListFields(cat_elev):
    keyval = field.name
    if keyval in elevDict:
        newname = elevDict[keyval][0]
        newalias = elevDict[keyval][0]
        print (keyval, newname)
        arcpy.AlterField_management(cat_elev, keyval, newname, newalias)

# Rename fields for slope tables
for field in arcpy.ListFields(wtd_slope):
    keyval = field.name
    if keyval in slopeDict:
        newname = slopeDict[keyval][1]
        newalias = slopeDict[keyval][1]
        print (keyval, newname)
        arcpy.AlterField_management(wtd_slope, keyval, newname, newalias)

for field in arcpy.ListFields(cat_slope):
    keyval = field.name
    if keyval in slopeDict:
        newname = slopeDict[keyval][0]
        newalias = slopeDict[keyval][0]
        print (keyval, newname)
        arcpy.AlterField_management(cat_slope, keyval, newname, newalias)

In [ ]:
# # Export copies of dbf tables as csv
# for table in outtables:
#     tablename = arcpy.Describe(table).basename + ".csv"
#     tablepath = os.path.join(outdir,tablename)
#     print( tablepath)
#     arcpy.conversion.TableToTable(table, outdir, tablename)

In [ ]:
import pandas as pd
pd.options.display.float_format = '{:.2f}'.format # only display 2 decimal places
# list to store covariate data frames
dfs = []

In [ ]:
# Make catchment elev df
cat_df = pd.DataFrame()
cat_field_list = []
for field in arcpy.ListFields(cat_elev):
    cat_field_list.append(field.name)
cat_elev_arr = arcpy.da.TableToNumPyArray(cat_elev,cat_field_list)
cat_df = pd.DataFrame(cat_elev_arr)
cat_df = cat_df.drop(["OBJECTID","NhdAwcH12_cat_elev_ZONE_CODE"],axis=1)
cat_df = cat_df.set_index('cat_ID_con')
dfs.append(cat_df)
cat_df


In [ ]:
# Make catchment slope df
cat_sl_df = pd.DataFrame()
cat_sl_field_list = []
for field in arcpy.ListFields(cat_slope):
    cat_sl_field_list.append(field.name)
cat_sl_arr = arcpy.da.TableToNumPyArray(cat_slope, cat_sl_field_list)
cat_sl_df = pd.DataFrame(cat_sl_arr)
cat_sl_df = cat_sl_df.drop(["OBJECTID", "NhdAwcH12_cat_slope_ZONE_CODE"],axis=1)
cat_sl_df = cat_sl_df.set_index('cat_ID_con')
dfs.append(cat_sl_df)
cat_sl_df

In [ ]:
# Make watershed elev df
wtd_df = pd.DataFrame()
wtd_field_list = []
for field in arcpy.ListFields(wtd_elev):
    wtd_field_list.append(field.name)
wtd_elev_arr = arcpy.da.TableToNumPyArray(wtd_elev,wtd_field_list)
wtd_df = pd.DataFrame(wtd_elev_arr)
wtd_df = wtd_df.drop(["OBJECTID","NhdAwcH12_wtd_elev_ZONE_CODE"],axis=1)
wtd_df = wtd_df.set_index('cat_ID_con')
dfs.append(wtd_df)
wtd_df

In [ ]:
# Make watershed slope df
wtd_sl_df = pd.DataFrame()
wtd_sl_field_list = []
for field in arcpy.ListFields(wtd_slope):
    wtd_sl_field_list.append(field.name)
wtd_sl_arr = arcpy.da.TableToNumPyArray(wtd_slope, wtd_sl_field_list)
wtd_sl_df = pd.DataFrame(wtd_sl_arr)
wtd_sl_df = wtd_sl_df.drop(["OBJECTID", "NhdAwcH12_wtd_slope_ZONE_CODE"],axis=1)
wtd_sl_df = wtd_sl_df.set_index('cat_ID_con')
dfs.append(wtd_sl_df)
wtd_sl_df

In [ ]:
# Make watershed north df
wtd_n_df = pd.DataFrame()
wtd_n_field_list = []
for field in arcpy.ListFields(wtd_per_north):
    wtd_n_field_list.append(field.name)
wtd_n_arr = arcpy.da.TableToNumPyArray(wtd_per_north,wtd_n_field_list)
wtd_n_df = pd.DataFrame(wtd_n_arr)
wtd_n_df = wtd_n_df.drop("OBJECTID",axis=1)
wtd_n_df = wtd_n_df.set_index('cat_ID_con')
dfs.append(wtd_n_df)
wtd_n_df

In [ ]:
# Make watershed wetland df
wtd_wet_df = pd.DataFrame()
wtd_wet_field_list = []
for field in arcpy.ListFields(wtd_wet):
    wtd_wet_field_list.append(field.name)
wtd_wet_arr = arcpy.da.TableToNumPyArray(wtd_wet,wtd_wet_field_list)
wtd_wet_df = pd.DataFrame(wtd_wet_arr)
wtd_wet_df = wtd_wet_df.drop("OBJECTID",axis=1)
wtd_wet_df = wtd_wet_df.set_index('cat_ID_con')
dfs.append(wtd_wet_df)
wtd_wet_df

In [ ]:
# Make watershed lakes df
wtd_lp_df = pd.DataFrame()
wtd_lp_field_list = []
for field in arcpy.ListFields(wtd_lp):
    wtd_lp_field_list.append(field.name)
wtd_lp_arr = arcpy.da.TableToNumPyArray(wtd_lp, wtd_lp_field_list)
wtd_lp_df = pd.DataFrame(wtd_lp_arr)
wtd_lp_df = wtd_lp_df.drop("OBJECTID",axis=1)
wtd_lp_df = wtd_lp_df.set_index('cat_ID_con')
dfs.append(wtd_lp_df)
wtd_lp_df

In [ ]:
# Make watershed glacier df
wtd_glac_df = pd.DataFrame()
wtd_glac_field_list = []
for field in arcpy.ListFields(wtd_glac):
    wtd_glac_field_list.append(field.name)
wtd_glac_arr = arcpy.da.TableToNumPyArray(wtd_glac, wtd_glac_field_list)
wtd_glac_df = pd.DataFrame(wtd_glac_arr)
wtd_glac_df = wtd_glac_df.drop("OBJECTID",axis=1)
wtd_glac_df = wtd_glac_df.set_index('cat_ID_con')
dfs.append(wtd_glac_df)
wtd_glac_df

## Merge all covariate dataframes together and drop unnecessary columns
 * Recalculate cat_ID as float64 type
 * Reorder columns
 * Export final csv


In [ ]:
# Merge all data frames together
import numpy as np
from functools import reduce
df_final = reduce(lambda left,right: pd.merge(left,right,on='cat_ID_con',how="outer"), dfs)
#Generate unique column names
def uniquify(df_final):
    seen = set()
    for item in df_final:
        fudge = 1
        newitem = item
        while newitem in seen:
            fudge += 1
            newitem = "{}_{}".format(item, fudge)
        yield newitem
        seen.add(newitem)
df_final.columns = list(uniquify(df_final))
#List of final columns in the order to output
# final_cols_old = ['cat_ID_txt','cat_ID','region', 'NhdAwcH12_cat_slope_COUNT', 'NhdAwcH12_cat_slope_AREA', 'NhdAwcH12_cat_slope_MIN', 'NhdAwcH12_cat_slope_MAX',
#               'NhdAwcH12_cat_slope_RANGE','NhdAwcH12_cat_slope_MEAN', 'NhdAwcH12_cat_slope_STD', 'NhdAwcH12_cat_slope_SUM', 'NhdAwcH12_cat_slope_MEDIAN', 'NhdAwcH12_cat_slope_PCT90',
#               'NhdAwcH12_cat_elev_COUNT', 'NhdAwcH12_cat_elev_AREA', 'NhdAwcH12_cat_elev_MIN', 'NhdAwcH12_cat_elev_MAX', 'NhdAwcH12_cat_elev_RANGE', 'NhdAwcH12_cat_elev_MEAN', 'NhdAwcH12_cat_elev_STD',
#               'NhdAwcH12_cat_elev_SUM', 'NhdAwcH12_cat_elev_VARIETY', 'NhdAwcH12_cat_elev_MAJORITY', 'NhdAwcH12_cat_elev_MINORITY', 'NhdAwcH12_cat_elev_MEDIAN', 'NhdAwcH12_cat_elev_PCT90',
#               'NhdAwcH12_wtd_elev_COUNT', 'NhdAwcH12_wtd_elev_AREA', 'NhdAwcH12_wtd_elev_MIN', 'NhdAwcH12_wtd_elev_MAX', 'NhdAwcH12_wtd_elev_RANGE', 'NhdAwcH12_wtd_elev_MEAN',
#               'NhdAwcH12_wtd_elev_STD', 'NhdAwcH12_wtd_elev_SUM', 'NhdAwcH12_wtd_elev_VARIETY', 'NhdAwcH12_wtd_elev_MAJORITY', 'NhdAwcH12_wtd_elev_MINORITY',
#               'NhdAwcH12_wtd_elev_MEDIAN', 'NhdAwcH12_wtd_elev_PCT90', 'NhdAwcH12_wtd_slope_COUNT', 'NhdAwcH12_wtd_slope_AREA', 'NhdAwcH12_wtd_slope_MIN', 'NhdAwcH12_wtd_slope_MAX',
#               'NhdAwcH12_wtd_slope_RANGE', 'NhdAwcH12_wtd_slope_MEAN', 'NhdAwcH12_wtd_slope_STD', 'NhdAwcH12_wtd_slope_SUM', 'NhdAwcH12_wtd_slope_MEDIAN', 'NhdAwcH12_wtd_slope_PCT90',
#               'NhdAwcH12_non_north_area', 'NhdAwcH12_north_area', 'NhdAwcH12_wtd_north_per', 'non_wetland_area', 'NhdAwcH12_wetland_area', 'NhdAwcH12_wtd_wet_per',
#               'NhdAwcH12_wtd_lake_area_sqm', 'NhdAwcH12_wtd_lake_per', 'NhdAwcH12_wtd_glacier_area_sqm', 'NhdAwcH12_wtd_glacier_per' ]
final_cols = ['cat_ID_txt','cat_ID','region', 'NhdAwcH12_cat_slope_COUNT', 'NhdAwcH12_cat_slope_AREA', 'NhdAwcH12_cat_slope_MIN', 'NhdAwcH12_cat_slope_MAX',
              'NhdAwcH12_cat_slope_RANGE','NhdAwcH12_cat_slope_MEAN', 'NhdAwcH12_cat_slope_STD', 'NhdAwcH12_cat_slope_SUM', 'NhdAwcH12_cat_slope_MEDIAN', 'NhdAwcH12_cat_slope_PCT90',
              'NhdAwcH12_cat_elev_COUNT', 'NhdAwcH12_cat_elev_AREA', 'NhdAwcH12_cat_elev_MIN', 'NhdAwcH12_cat_elev_MAX', 'NhdAwcH12_cat_elev_RANGE', 'NhdAwcH12_cat_elev_MEAN', 'NhdAwcH12_cat_elev_STD',
              'NhdAwcH12_cat_elev_SUM', 'NhdAwcH12_cat_elev_VARIETY', 'NhdAwcH12_cat_elev_MAJORITY', 'NhdAwcH12_cat_elev_MINORITY', 'NhdAwcH12_cat_elev_MEDIAN', 'NhdAwcH12_cat_elev_PCT90',
              'NhdAwcH12_wtd_elev_MIN', 'NhdAwcH12_wtd_elev_MAX','NhdAwcH12_wtd_elev_MEAN','NhdAwcH12_wtd_elev_STD','NhdAwcH12_wtd_slope_MIN', 'NhdAwcH12_wtd_slope_MAX','NhdAwcH12_wtd_slope_MEAN',
              'NhdAwcH12_wtd_slope_STD','NhdAwcH12_non_north_area', 'NhdAwcH12_north_area', 'NhdAwcH12_wtd_north_per', 'non_wetland_area', 'NhdAwcH12_wetland_area', 'NhdAwcH12_wtd_wet_per',
              'NhdAwcH12_wtd_lake_area_sqm', 'NhdAwcH12_wtd_lake_per', 'NhdAwcH12_wtd_glacier_area_sqm', 'NhdAwcH12_wtd_glacier_per' ]

#Create list of duplicate column names and drop
drop_cols = ['cat_ID_txt_y', 'region_y', 'cat_ID_txt_x_2', 'region_x_2', 'region_y_2', 'cat_ID_txt_y_2', 'region_x_3',
             'cat_ID_txt_x_3', 'cat_ID_txt_y_3', 'FType', 'region_y_3','cat_ID_txt_x_4', 'O1Region', 'region_x_4',
             'cat_ID_y', 'cat_ID_txt_y_4', 'region_y_4']
df_final.drop(columns=drop_cols, axis = 1, inplace=True)
#rename columns
df_final.rename({'cat_ID_txt_x':'cat_ID_txt','cat_ID_x':'cat_ID','region_x':'region'},axis=1, inplace=True)
#Recalculate cat_ID
df_final['cat_ID'] = df_final['cat_ID_txt'].astype(np.float64)
# reorder cols
df_final = df_final.reindex(columns=final_cols)
df_final

In [ ]:
# Export merged dataframe to csv
cov_csv_out = os.path.join(outdir,'AKSSF_AWC_HUC12s_Covariates.csv')
df_final.to_csv(cov_csv_out, encoding = 'utf-8')
print('Export all covariates dataframe to csv complete')


In [ ]:
# bbay_df = df_final.filter(like='Bristol_Bay', axis = 0)
# bbay_df

In [ ]:
# # kod_df = df_final.filter(like='Kodiak', axis = 0)
# kod_df

In [ ]:
# pws_df = df_final.filter(like='Prince', axis = 0)
# pws_df

In [ ]:
# ci_df = df_final.filter(like='Cook', axis = 0)
# ci_df

In [ ]:
cop_df = df_final.filter(like='Copper', axis = 0)
cop_df

In [ ]:
ci_df = df_final.filter(like='Cook', axis = 0)
ci_df